See eventual readme and explanation page for full comments and motivation on this project.

#### import statements

In [429]:
from trueskill import Rating, quality_1vs1, rate_1vs1, rate, TrueSkill
import trueskill

import pandas as pd
import numpy as np

import numpy.linalg as la
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import json
import requests
import re
import os
import itertools
import bs4
import networkx
import urllib.request
from itertools import combinations
from bs4 import BeautifulSoup
import collections
from collections import Counter

import time

import warnings
warnings.filterwarnings(action='ignore')

from functools import reduce

from datetime import date
from dateutil.relativedelta import relativedelta

In [430]:
#setting today's date for export statements

today = date.today()
today = str(today.strftime("%Y")) + str(today.strftime("%m")) + str(today.strftime("%d"))
today

'20210511'

#### custom functions for parsing scraped strings

In [431]:
def match_parser(item):
    '''
    categorizes matches by number of participants and type for parsing and strips text from string
    '''
    matchguide = pd.read_csv('aew_matchguide.csv') #document i made of match types to strip that out from the text
    for i, j in matchguide.iterrows():
        if item[:j.len] == j.specific:
            item = item[j.len:]
            return item, j.type
            break
    return item, 'uncategorized'

times = []

for i in range(0, 60):
    '''converts listed matchtimes into separate variable in case i want to do something with that one day'''
    for j in range(0,60):
        if j < 10:
            times.append("("+str(i)+":0"+str(j)+")")
        else:
            times.append("("+str(i)+":"+str(j)+")")

### All below functions split matches into winners and losers depending on the match structure           
            
def singles_splitter(item):
    '''
    
    '''
    check = re.split('defeats', item)
    check = [item.rstrip().lstrip() for item in check]
    for item in check:
        if any(tme in item for tme in times):
            match_len = item[-7:]
            fixed = item[:-7]
            del check[-1]
            check.append(fixed)
            check.append(match_len.rstrip().lstrip())
            break
    return check

def multiway_splitter(item):
    check = re.split(' and ', item)
    check = [item.rstrip().lstrip() for item in check]
    for item in check:
        if any(tme in item for tme in times):
            match_len = item[-7:]
            fixed = item[:-7]
            del check[-1]
            check.append(fixed)
            check.append(match_len.rstrip().lstrip())
            break
    return check

def tag_splitter(item):
    check = re.split('defeat', item)
    check = [item.rstrip().lstrip() for item in check]
    for item in check:
        if any(tme in item for tme in times):
            match_len = item[-7:]
            fixed = item[:-7]
            del check[-1]
            check.append(fixed)
            check.append(match_len.rstrip().lstrip())
            break
    return check

def tag_team_parser(i):
    if re.search("\(", i):
        i = i.replace('(', ')')
        i = i.split(')')
        return(i[1])
    else:
        return(i)
    
def notes_stripper(item):
    '''removes misc bugs and issues'''
    item = item.replace(r" by referee's decision","")
    item = item.replace(r" by referee's decision","")
    item = item.replace(r" by Count Out","")
    item = item.replace(r" by Count Out","")
    item = item.replace(r" [2:1] - Kenny Omega pins PAC [2:1]","")
    return item

#### scraping cagematch for AEW events

In [432]:
webpage = 'https://www.cagematch.net/?id=8&nr=2287&page=4&s='
pages = ['0', '100', '200', '300']

all_events = []

for page in pages:
    cagematch_html = requests.get(str(webpage + page), headers={'Accept-Encoding': 'identity'})
    cagematch_html.encoding = cagematch_html.apparent_encoding
    soup = BeautifulSoup(cagematch_html.text, features='html.parser')
    links = []

    for link in soup.find_all('a'):
        links.append(link.get('href'))
        
    links2 = [item for item in links if item and '?id=1&' in item]
    links2 = [item for item in links2 if item and '&page=2' not in item]
    links2 = [item for item in links2 if item and 'view' not in item]
    for item in links2:
        all_events.append(item)

In [433]:
#test

webpage = 'https://www.cagematch.net/?id=1&nr=283430'

cagematch_html = requests.get(webpage, headers={'Accept-Encoding': 'identity'})
cagematch_html.encoding = cagematch_html.apparent_encoding
soup = BeautifulSoup(cagematch_html.text, features='html.parser')

soup.get_text()

'\n\n\n\n\n\n\n\n\n\n\nAEW Dynamite #45 « Events Database « CAGEMATCH - The Internet Wrestling Database\n\nwindow.onload = function() { loadComments("commentBox", "1", "283430", "", "en"); }\n\n\n\nNot logged in or registered. | Log In | Register | Password lost?\n\n\n\n\n\nAbout UsContactCAGEMATCH TeamYouTubeTwitterPatreon\n\nWrestlersOverviewWrestlersGimmicksTop ListsCommentsChangesEventsOverviewResultsTV + PPVUpcoming ShowsTop ListsSearch For EventsCommentsChangesMatchesOverviewMatchguideTop ListsSearch For EventsStatisticsCommentsChangesPromotionsOverviewPromotionsPromotion NamesTop ListsCommentsChangesTitlesOverviewTitle NamesTitle ChangesTop ListsCommentsChangesDatabasesTag TeamsStablesPromosFeudsRivalriesTournamentsAwardsBooksDVDsWrestling GamesMoviesThemesOtherSOTWArticlesPodcastsDocumentariesNewsboardChanges TrackerRandom ItemRatings CenterRatings System RulesHelp\n\n\n\n\n\n\nCAGEMATCH » \nEvents Database » \nAEW Dynamite #45\n\n\nAEW Dynamite #45TV-ShowResultsCardCard with g

#### Initial parse of results

In [434]:
event_titles = []
match_list = []

event_card = pd.DataFrame(columns=['Event', 'Date', 'Card'])

f = open("all_events_external.txt", "r")
parsed_events = f.read()
f.close()

parsed_events = parsed_events.split(',')[:-1]

print(parsed_events)

for page in all_events:
    if page in parsed_events:
        print('already parsed:' + page)
        continue
    else: 
        time.sleep(2)
        webpage = 'https://www.cagematch.net/' + page

        cagematch_html = requests.get(webpage, headers={'Accept-Encoding': 'identity'})
        cagematch_html.encoding = cagematch_html.apparent_encoding
        soup = BeautifulSoup(cagematch_html.text, features='html.parser')

        text = soup.get_text()

        x = re.split('::::', text)

        z = re.split('«', x[0])
        event = z[0].replace('\n', '')
        event = event.rstrip()

        if event != 'Card':
            print('now parsing:' + page)
            try:
                date = re.search('Broadcast date\:(\d\d.\d\d.\d\d\d\d)', z[2]).group(1)
            except AttributeError:
                date = re.search('Date\:(\d\d.\d\d.\d\d\d\d)', z[2]).group(1)

            event_titles.append(event)
            print(event)
            print(date)

            y = [item for item in x if 'defeat' in item] #DoubleDQ, DQ, time limit draw

            sub_list = ['(c)', ' - TITLE CHANGE !!!', '!!!', 'TITLE CHANGE']

            for sub in sub_list:
                y = [item.replace(sub, '') for item in y]

            splits = ['All workers', 'NotesThis text is hidden', 'Info:']

            for split in splits:
                y = [item.split(split, 1)[0] for item in y]

            for item in y:
                match_list.append(item)
                print(item)

            event_card = event_card.append({'Event': event, 'Date': date, 'Card': y}, ignore_index=True)
            
            f = open("all_events_external.txt", "a")
            f.write(page + ',')
            f.close()
        else:
            print('future event:' + page)

['?id=1&nr=309228', '?id=1&nr=310297', '?id=1&nr=310298', '?id=1&nr=309150', '?id=1&nr=309374', '?id=1&nr=309844', '?id=1&nr=308648', '?id=1&nr=306901', '?id=1&nr=308860', '?id=1&nr=253470', '?id=1&nr=308859', '?id=1&nr=255678', '?id=1&nr=305558', '?id=1&nr=308588', '?id=1&nr=253471', '?id=1&nr=307506', '?id=1&nr=308122', '?id=1&nr=305701', '?id=1&nr=306810', '?id=1&nr=307320', '?id=1&nr=305674', '?id=1&nr=306524', '?id=1&nr=306653', '?id=1&nr=305454', '?id=1&nr=305624', '?id=1&nr=306008', '?id=1&nr=296143', '?id=1&nr=305007', '?id=1&nr=302689', '?id=1&nr=305116', '?id=1&nr=305502', '?id=1&nr=304813', '?id=1&nr=304018', '?id=1&nr=303769', '?id=1&nr=300951', '?id=1&nr=304680', '?id=1&nr=300950', '?id=1&nr=303870', '?id=1&nr=303609', '?id=1&nr=300949', '?id=1&nr=303336', '?id=1&nr=300948', '?id=1&nr=303008', '?id=1&nr=300947', '?id=1&nr=302187', '?id=1&nr=300498', '?id=1&nr=301607', '?id=1&nr=254325', '?id=1&nr=301056', '?id=1&nr=298900', '?id=1&nr=300426', '?id=1&nr=299650', '?id=1&nr=3

In [435]:
event_card.to_csv('AEW_export_' + today + '.csv', index=False)

In [436]:
#event_card = pd.read_csv('AEW_export_021121.csv')
#event_card = event_card.drop(columns=['Unnamed: 0'])

In [437]:
#https://stackoverflow.com/questions/27263805/pandas-column-of-lists-create-a-row-for-each-list-element

lst_col = 'Card'

r = pd.DataFrame({
      col:np.repeat(event_card[col].values, event_card[lst_col].str.len())
      for col in event_card.columns.drop(lst_col)}
    ).assign(**{lst_col:np.concatenate(event_card[lst_col].values)})[event_card.columns]


r

,Event,Date,Card
0,AEW Dark: Elevation #9,10.05.2021,Singles MatchLee Johnson defeats David Ali
1,AEW Dark: Elevation #9,10.05.2021,Singles MatchEddie Kingston defeats VSK
2,AEW Dark: Elevation #9,10.05.2021,Tag Team MatchEthan Page & Scorpio Sky defeat ...
3,AEW Dark: Elevation #9,10.05.2021,Singles MatchThunder Rosa defeats Renee Michelle
4,AEW Dark: Elevation #9,10.05.2021,"Singles MatchChuck Taylor (w/Kris Statlander, ..."
5,AEW Dark: Elevation #9,10.05.2021,Singles MatchLeyla Hirsch defeats Dani Jordyn
6,AEW Dark: Elevation #9,10.05.2021,Singles MatchMatt Sydal defeats Manny Smith
7,AEW Dark: Elevation #9,10.05.2021,Singles MatchJade Cargill defeats Reka Tehaka
8,AEW Dark: Elevation #9,10.05.2021,Tag Team MatchThe Hardy Family Office (Marq Qu...
9,AEW Dark: Elevation #9,10.05.2021,Singles MatchDr. Britt Baker DMD (w/Rebel) def...


In [438]:
y = [match_parser(item) for item in r.Card]

temp1 = pd.DataFrame(columns=['results', 'matchtype'], data = y)

r['result'] = temp1.results
r['matchtype'] = temp1.matchtype

r

,Event,Date,Card,result,matchtype
0,AEW Dark: Elevation #9,10.05.2021,Singles MatchLee Johnson defeats David Ali,Lee Johnson defeats David Ali,singles
1,AEW Dark: Elevation #9,10.05.2021,Singles MatchEddie Kingston defeats VSK,Eddie Kingston defeats VSK,singles
2,AEW Dark: Elevation #9,10.05.2021,Tag Team MatchEthan Page & Scorpio Sky defeat ...,Ethan Page & Scorpio Sky defeat Baron Black & ...,tag
3,AEW Dark: Elevation #9,10.05.2021,Singles MatchThunder Rosa defeats Renee Michelle,Thunder Rosa defeats Renee Michelle,singles
4,AEW Dark: Elevation #9,10.05.2021,"Singles MatchChuck Taylor (w/Kris Statlander, ...","Chuck Taylor (w/Kris Statlander, Orange Cassid...",singles
5,AEW Dark: Elevation #9,10.05.2021,Singles MatchLeyla Hirsch defeats Dani Jordyn,Leyla Hirsch defeats Dani Jordyn,singles
6,AEW Dark: Elevation #9,10.05.2021,Singles MatchMatt Sydal defeats Manny Smith,Matt Sydal defeats Manny Smith,singles
7,AEW Dark: Elevation #9,10.05.2021,Singles MatchJade Cargill defeats Reka Tehaka,Jade Cargill defeats Reka Tehaka,singles
8,AEW Dark: Elevation #9,10.05.2021,Tag Team MatchThe Hardy Family Office (Marq Qu...,The Hardy Family Office (Marq Quen & Matt Hard...,tag
9,AEW Dark: Elevation #9,10.05.2021,Singles MatchDr. Britt Baker DMD (w/Rebel) def...,Dr. Britt Baker DMD (w/Rebel) defeats Raychell...,singles


In [439]:
r_master = pd.read_csv('AEW_export_r_master.csv')
r_master

,Event,Date,Card,result,matchtype
0,AEW Dynamite #84 - Blood & Guts,05.05.2021,Tag Team MatchEddie Kingston & Jon Moxley defe...,Eddie Kingston & Jon Moxley defeat Kenny Omega...,tag
1,AEW Dynamite #84 - Blood & Guts,05.05.2021,Singles MatchCody Rhodes (w/Arn Anderson) defe...,Cody Rhodes (w/Arn Anderson) defeats QT Marsha...,singles
2,AEW Dynamite #84 - Blood & Guts,05.05.2021,Singles MatchDr. Britt Baker DMD (w/Rebel) def...,Dr. Britt Baker DMD (w/Rebel) defeats Julia Ha...,singles
3,AEW Dynamite #84 - Blood & Guts,05.05.2021,AEW World Tag Team Title #1 Contendership Four...,SCU (Christopher Daniels & Frankie Kazarian) d...,fourwaytag
4,AEW Dynamite #84 - Blood & Guts,05.05.2021,Blood And Guts Ten Man Tag Team MatchThe Pinna...,"The Pinnacle (Cash Wheeler, Dax Harwood, MJF, ...",tag10
...,...,...,...,...,...
1432,AEW Double Or Nothing 2019,25.05.2019,Singles MatchCody (w/Brandi Rhodes) defeats Du...,Cody (w/Brandi Rhodes) defeats Dustin Rhodes (...,singles
1433,AEW Double Or Nothing 2019,25.05.2019,AAA World Tag Team Title MatchThe Young Bucks ...,The Young Bucks (Matt Jackson & Nick Jackson) ...,tag
1434,AEW Double Or Nothing 2019,25.05.2019,Singles MatchChris Jericho defeats Kenny Omega...,Chris Jericho defeats Kenny Omega (26:52),singles
1435,AEW Double Or Nothing - The Buy In,25.05.2019,21 Man Casino Battle Royale MatchAdam Page def...,Adam Page defeats Ace Romero and Billy Gunn an...,br


In [440]:
r = pd.concat([r, r_master])
r.to_csv('AEW_export_r_master.csv', index=False)

#### categorizing matches

In [441]:
r = pd.read_csv('AEW_export_r_master.csv')

In [442]:
r[r.matchtype=='uncategorized']

,Event,Date,Card,result,matchtype
83,AEW The House Always Wins,09.04.2021,AEW TNT Title #1 Contendership 15 Man Battle R...,AEW TNT Title #1 Contendership 15 Man Battle R...,uncategorized


In [443]:
r.matchtype.value_counts()

singles          844
tag              438
tag6              87
tag8              24
br                10
fourway            9
threewaytag        8
threeway           7
tag10              6
fourwaytag         5
unsanctioned       4
handicap2on1       3
tag12              2
uncategorized      1
handicap3on1       1
sixway             1
nineway            1
Name: matchtype, dtype: int64

In [444]:
r = r[['Event', 'Date', 'result', 'matchtype']]

#### parsing singles matches

In [445]:
singles_matches = r[r['matchtype'] == 'singles']

singles_matches['win1'] = [singles_splitter(i)[0] for i in singles_matches['result']]
singles_matches['lose1'] = [singles_splitter(i)[1] for i in singles_matches['result']]
singles_matches['time'] = [singles_splitter(i)[2] if len(singles_splitter(i)) == 3 else '(Unk.)' for i in singles_matches['result']]

singles_matches.win1 = [i.rstrip().lstrip() for i in singles_matches['win1'].str.replace(r"\(w\/.*?\)","")]
singles_matches.lose1 = [i.rstrip().lstrip() for i in singles_matches['lose1'].str.replace(r"\(w\/.*?\)","")]
singles_matches.lose1 = [notes_stripper(i) for i in singles_matches.lose1]

In [446]:
singles_matches[:20]

,Event,Date,result,matchtype,win1,lose1,time
0,AEW Dark: Elevation #9,10.05.2021,Lee Johnson defeats David Ali,singles,Lee Johnson,David Ali,(Unk.)
1,AEW Dark: Elevation #9,10.05.2021,Eddie Kingston defeats VSK,singles,Eddie Kingston,VSK,(Unk.)
3,AEW Dark: Elevation #9,10.05.2021,Thunder Rosa defeats Renee Michelle,singles,Thunder Rosa,Renee Michelle,(Unk.)
4,AEW Dark: Elevation #9,10.05.2021,"Chuck Taylor (w/Kris Statlander, Orange Cassid...",singles,Chuck Taylor,Vary Morales,(Unk.)
5,AEW Dark: Elevation #9,10.05.2021,Leyla Hirsch defeats Dani Jordyn,singles,Leyla Hirsch,Dani Jordyn,(Unk.)
6,AEW Dark: Elevation #9,10.05.2021,Matt Sydal defeats Manny Smith,singles,Matt Sydal,Manny Smith,(Unk.)
7,AEW Dark: Elevation #9,10.05.2021,Jade Cargill defeats Reka Tehaka,singles,Jade Cargill,Reka Tehaka,(Unk.)
9,AEW Dark: Elevation #9,10.05.2021,Dr. Britt Baker DMD (w/Rebel) defeats Raychell...,singles,Dr. Britt Baker DMD,Raychell Rose,(Unk.)
10,AEW Dark: Elevation #9,10.05.2021,Tay Conti defeats Madi Wrenkowski,singles,Tay Conti,Madi Wrenkowski,(Unk.)
12,AEW Dark: Elevation #9,10.05.2021,Alex Reynolds (w/Colt Cabana & John Silver) de...,singles,Alex Reynolds,Isiah Kassidy,(Unk.)


#### parsing tag matches

In [447]:
tag_matches = r[r['matchtype'] == 'tag']

tag_matches['win1'] = [tag_splitter(i)[0] for i in tag_matches['result']]
tag_matches['lose1'] = [tag_splitter(i)[1] for i in tag_matches['result']]
tag_matches['time'] = [tag_splitter(i)[2] if len(tag_splitter(i)) == 3 else '(Unk.)' for i in tag_matches['result']]

tag_matches.win1 = [i.rstrip().lstrip() for i in tag_matches['win1'].str.replace(r"\(w\/.*?\)","")]
tag_matches.lose1 = [i.rstrip().lstrip() for i in tag_matches['lose1'].str.replace(r"\(w\/.*?\)","")]

tag_matches['win1'] = [tag_team_parser(i) for i in tag_matches.win1]
tag_matches['lose1'] = [tag_team_parser(i) for i in tag_matches.lose1]

tag_matches[['win1','win2']] = tag_matches.win1.str.split(" & ",expand=True)
tag_matches[['lose1','lose2']] = tag_matches.lose1.str.split(" & ",expand=True) 

tag_matches = tag_matches[['Event', 'Date', 'result', 'matchtype', 'win1', 'win2', 'lose1', 'lose2', 'time']]

In [448]:
tag_matches

,Event,Date,result,matchtype,win1,win2,lose1,lose2,time
2,AEW Dark: Elevation #9,10.05.2021,Ethan Page & Scorpio Sky defeat Baron Black & ...,tag,Ethan Page,Scorpio Sky,Baron Black,Fuego Del Sol,(Unk.)
8,AEW Dark: Elevation #9,10.05.2021,The Hardy Family Office (Marq Quen & Matt Hard...,tag,Marq Quen,Matt Hardy,Dean Alexander,Dillon McQueen,(Unk.)
11,AEW Dark: Elevation #9,10.05.2021,Jurassic Express (Jungle Boy & Marko Stunt) de...,tag,Jungle Boy,Marko Stunt,Luther,Serpentico,(Unk.)
14,AEW Dynamite #84 - Blood & Guts,05.05.2021,Eddie Kingston & Jon Moxley defeat Kenny Omega...,tag,Eddie Kingston,Jon Moxley,Kenny Omega,MT Nakazawa,(8:03)
24,AEW Dark: Elevation #8,03.05.2021,FTR (Cash Wheeler & Dax Harwood) (w/Tully Blan...,tag,Cash Wheeler,Dax Harwood,Terrell Hughes,Terrence Hughes,(4:58)
...,...,...,...,...,...,...,...,...,...
1429,AEW Fight For The Fallen,13.07.2019,The Lucha Brothers (Pentagon Jr. & Rey Fenix) ...,tag,Pentagon Jr.,Rey Fenix,Frankie Kazarian,Scorpio Sky,(15:05)
1431,AEW Fight For The Fallen,13.07.2019,The Young Bucks (Matt Jackson & Nick Jackson) ...,tag,Matt Jackson,Nick Jackson,Cody,Dustin Rhodes,(31:26)
1433,AEW Fight For The Fallen - The Buy In,13.07.2019,Bea Priestley & Shoko Nakajima defeat Dr. Brit...,tag,Bea Priestley,Shoko Nakajima,Dr. Britt Baker DMD,Riho,(15:38)
1444,AEW Double Or Nothing 2019,25.05.2019,Best Friends (Chuck Taylor & Trent) defeat Ang...,tag,Chuck Taylor,Trent,Angelico,Jack Evans,(12:42)


#### parsing tag6s

In [449]:
def tag6_parser(item):
    one_and_two = '.*? & .*? \(.*\)'
    two_and_one = '.*? \(.*\) & .*'
    trio = '.*? \(.*\)'
    three_singles = '.*? \& .*'
    
    if re.match(one_and_two, item):
        x = re.split(r'[,&()]\s*', item)
        z = [x[0], x[2], x[3]]
        return(z)
    elif re.match(two_and_one, item):
        x = re.split(r'[,&()]\s*', item)
        z = [x[1], x[2], x[4]]
        return(z)
    elif re.match(trio, item):
        x = re.split(r'[,&()]\s*', item)
        z = [x[1], x[2], x[3]]
        return(z)
    elif re.match(three_singles, item):
        x = re.split(r'[,&()]\s*', item)
        z = [x[0], x[1], x[2]]
        return(z)

In [450]:
tag6_matches = r[r['matchtype'] == 'tag6']

tag6_matches['win'] = [tag_splitter(i)[0] for i in tag6_matches['result']]
tag6_matches['lose'] = [tag_splitter(i)[1] for i in tag6_matches['result']]
tag6_matches['time'] = [tag_splitter(i)[2] if len(tag_splitter(i)) == 3 else '(Unk.)' for i in tag6_matches['result']]

tag6_matches.win = [i.rstrip().lstrip() for i in tag6_matches['win'].str.replace(r"\(w.*\)","")]
tag6_matches.lose = [i.rstrip().lstrip() for i in tag6_matches['lose'].str.replace(r"\(w.*\)","")]

tag6_matches.win = [tag6_parser(i) for i in tag6_matches.win]
tag6_matches.lose = [tag6_parser(i) for i in tag6_matches.lose]

tag6_matches['win1'] = [i[0] for i in tag6_matches.win]
tag6_matches['win2'] = [i[1] for i in tag6_matches.win]
tag6_matches['win3'] = [i[2] for i in tag6_matches.win]

tag6_matches['lose1'] = [i[0] for i in tag6_matches.lose]
tag6_matches['lose2'] = [i[1] for i in tag6_matches.lose]
tag6_matches['lose3'] = [i[2] for i in tag6_matches.lose]

tag6_matches = tag6_matches[['Event', 'Date', 'result', 'matchtype', 'win1', 'win2', 'win3', 'lose1', 'lose2', 'lose3', 'time']]


tag6_matches

,Event,Date,result,matchtype,win1,win2,win3,lose1,lose2,lose3,time
44,AEW Dark #87,04.05.2021,"The Dark Order (Alex Reynolds, Evil Uno & Stu ...",tag6,Alex Reynolds,Evil Uno,Stu Grayson,Cezar Bononi,JD Drake,Ryan Nemeth,(7:07)
51,AEW Dynamite #83,28.04.2021,"The Factory (Aaron Solow, Nick Comoroto & QT M...",tag6,Aaron Solow,Nick Comoroto,QT Marshall,Billy Gunn,Dustin Rhodes,Lee Johnson,(6:44)
76,AEW Dark #86,27.04.2021,"The Dark Order (Evil Uno, Stu Grayson & Ten) (...",tag6,Evil Uno,Stu Grayson,Ten,Isiah Kassidy,Marq Quen,The Blade,(10:07)
87,AEW The House Always Wins,09.04.2021,Best Friends (Chuck Taylor & Trent) & Orange C...,tag6,Chuck Taylor,Trent,Orange Cassidy,Angelico,Jack Evans,Max Caster,(11:10)
97,AEW Dark #85,20.04.2021,"The Dark Order (Evil Uno, Stu Grayson & Ten) (...",tag6,Evil Uno,Stu Grayson,Ten,Jay Lyon,Ken Broadway,Midas Black,(5:17)
...,...,...,...,...,...,...,...,...,...,...,...
1415,AEW All Out 2019,31.08.2019,"SCU (Christopher Daniels, Frankie Kazarian & S...",tag6,Christopher Daniels,Frankie Kazarian,Scorpio Sky,Jungle Boy,Luchasaurus,Marko Stunt,(11:40)
1425,AEW Fight For The Fallen,13.07.2019,"Maxwell Jacob Friedman, Sammy Guevara & Shawn ...",tag6,Maxwell Jacob Friedman,Sammy Guevara,Shawn Spears,Darby Allin,Jimmy Havoc,Joey Janela,(13:13)
1437,AEW Fyter Fest,29.06.2019,"The Elite (Kenny Omega, Matt Jackson & Nick Ja...",tag6,Kenny Omega,Matt Jackson,Nick Jackson,Laredo Kid,Pentagon Jr.,Rey Fenix,(20:29)
1442,AEW Double Or Nothing 2019,25.05.2019,"SCU (Christopher Daniels, Frankie Kazarian & S...",tag6,Christopher Daniels,Frankie Kazarian,Scorpio Sky,CIMA,El Lindaman,T-Hawk,(13:38)


#### parsing tag8s

In [451]:
def tag8_parser(item):
    two_and_two = '.*? \(.*\) & .*? \(.*\)'
    two_and_two_error = '.*? \(.*\)  & .*? \(.*\)'
    three_and_one = '.*? \(.*\) & .*'
    one_one_two = '.*,.* & .*? \(.*\)'
    two_one_one = '.*\(.*\),.*&.*'
    one_and_three = '.* & .*? \(.*\)'
    quad = '.*? \(.*\)'
    
    if re.match(two_and_two, item) or re.match(two_and_two_error, item):
        x = re.split(r'[,&()]\s*', item)
        z = [x[1], x[2], x[5], x[6]]
        return(z)
    elif re.match(two_one_one, item):
        x = re.split(r'[,&()]\s*', item)
        z = [x[1], x[2], x[4], x[5]]
        return(z)
    elif re.match(three_and_one, item):
        x = re.split(r'[,&()]\s*', item)
        z = [x[1], x[2], x[3], x[5]]
        return(z)
    elif re.match(one_one_two, item):
        x = re.split(r'[,&()]\s*', item)
        z = [x[0], x[1], x[3], x[4]]
        return(z)
    elif re.match(one_and_three, item):
        x = re.split(r'[,&()]\s*', item)
        z = [x[0], x[2], x[3], x[4]]
        return(z)
    elif re.match(quad, item):
        x = re.split(r'[,&()]\s*', item)
        z = [x[1], x[2], x[3], x[4]]
        return(z)
    else:
        x = re.split(r'[,&()]\s*', item)
        z = [x[0], x[1], x[2], x[3]]
        return(z)

In [452]:
tag8_matches = r[r['matchtype'] == 'tag8']

tag8_matches['win'] = [tag_splitter(i)[0] for i in tag8_matches['result']]
tag8_matches['lose'] = [tag_splitter(i)[1] for i in tag8_matches['result']]
tag8_matches['time'] = [tag_splitter(i)[2] if len(tag_splitter(i)) == 3 else '(Unk.)' for i in tag8_matches['result']]

tag8_matches.win = [i.rstrip().lstrip() for i in tag8_matches['win'].str.replace(r"\(w\/.*?\)","")]
tag8_matches.lose = [i.rstrip().lstrip() for i in tag8_matches['lose'].str.replace(r"\(w\/.*?\)","")]

tag8_matches.win = [tag8_parser(i) for i in tag8_matches.win]
tag8_matches.lose = [tag8_parser(i) for i in tag8_matches.lose]

tag8_matches['win1'] = [i[0] for i in tag8_matches.win]
tag8_matches['win2'] = [i[1] for i in tag8_matches.win]
tag8_matches['win3'] = [i[2] for i in tag8_matches.win]
tag8_matches['win4'] = [i[3] for i in tag8_matches.win]

tag8_matches['lose1'] = [i[0] for i in tag8_matches.lose]
tag8_matches['lose2'] = [i[1] for i in tag8_matches.lose]
tag8_matches['lose3'] = [i[2] for i in tag8_matches.lose]
tag8_matches['lose4'] = [i[3] for i in tag8_matches.lose]


tag8_matches = tag8_matches[['Event', 'Date', 'result', 'matchtype', 'win1', 'win2', 'win3', 'win4', 'lose1', 'lose2', 'lose3', 'lose4', 'time']]


tag8_matches


,Event,Date,result,matchtype,win1,win2,win3,win4,lose1,lose2,lose3,lose4,time
84,AEW The House Always Wins,09.04.2021,"The Pinnacle (Cash Wheeler, Dax Harwood, MJF &...",tag8,Cash Wheeler,Dax Harwood,MJF,Shawn Spears,Christopher Daniels,Dante Martin,Jungle Boy,Luchasaurus,(14:30)
89,AEW The House Always Wins,09.04.2021,"Hikaru Shida, Red Velvet, Ryo Mizunami & Tay C...",tag8,Hikaru Shida,Red Velvet,Ryo Mizunami,Tay Conti,Dr. Britt Baker DMD,Nyla Rose,Rebel,The Bunny,(14:00)
177,AEW Dark: Elevation #4,05.04.2021,The Butcher And The Blade (The Blade & The But...,tag8,The Blade,The Butcher,Isiah Kassidy,Marq Quen,Adam Priest,D3,Fuego Del Sol,Ryzin,(4:30)
272,AEW Dark: Elevation #1,15.03.2021,The Butcher And The Blade (The Blade & The But...,tag8,The Blade,The Butcher,Isiah Kassidy,Marq Quen,Brick Aldridge,Carlie Bravo,David Ali,Dean Alexander,(4:32)
281,AEW Dark #80,16.03.2021,"The Dark Order (Colt Cabana, Evil Uno, Five & ...",tag8,Colt Cabana,Evil Uno,Five,Stu Grayson,Angel Fashion,Baron Black,Ryzin,Vary Morales,(3:56)
321,AEW Dark #79,09.03.2021,"The Dark Order (Alex Reynolds, Colt Cabana, Ev...",tag8,Alex Reynolds,Colt Cabana,Evil Uno,Stu Grayson,Aaron Frye,D3,Jon Cruz,Vary Morales,(5:41)
377,AEW Dark #76,23.02.2021,"Bear Country (Bear Boulder & Bear Bronson), Jo...",tag8,Bear Boulder,Bear Bronson,Joey Janela,Sonny Kiss,Aaron Frye,Daniel Joseph,Levy Shapiro,M'Badu,(5:24)
425,AEW Dynamite #70,27.01.2021,The Good Brothers (Karl Anderson & Luke Gallow...,tag8,Karl Anderson,Luke Gallows,Matt Jackson,Nick Jackson,Alex Reynolds,Evil Uno,John Silver,Stu Grayson,(11:46)
433,AEW Dark #73,02.02.2021,"Bear Country (Bear Boulder & Bear Bronson), Jo...",tag8,Bear Boulder,Bear Bronson,Joey Janela,Sonny Kiss,Aaron Solow,Baron Black,Mike Verna,Shawn Dean,(5:15)
439,AEW Dynamite #69,20.01.2021,"Adam Page & The Dark Order (Alex Reynolds, Col...",tag8,Adam Page,Alex Reynolds,Colt Cabana,John Silver,Luther,Serpentico,Angelico,Jack Evans,(6:20)


#### parsing tag10s

In [453]:
def tag10_parser(item):
    four_and_one = '.*? \(.*\) & .*'
    three_one_one = '.*? \(.*\), .*? & .*'
    two_one_one_one = '.*? \(.*\), .*?, .*? & .*'
    two_two_one = '.*? \(.*\), .*? \(.*\) & .*'
    two_three = '.*? \(.*?\) & .*?\(.*?\)'
    fivesome = '.*? \(.*\)'
    one_two_two = '.*?, .*? \(.*\) &.*? \(.*\)'
    
    if re.match(two_two_one, item):
        x = re.split(r'[,&()]\s*', item)
        z = [x[1], x[2], x[5], x[6], x[8]]
        return(z)
    elif re.match(two_one_one_one, item):
        x = re.split(r'[,&()]\s*', item)
        z = [x[1], x[2], x[4], x[5], x[6]]
        return(z)
    elif re.match(one_two_two, item):
        x = re.split(r'[,&()]\s*', item)
        z = [x[0], x[2], x[3], x[6], x[7]]
        return(z)
    elif re.match(three_one_one, item):
        x = re.split(r'[,&()]\s*', item)
        z = [x[1], x[2], x[3], x[5], x[6]]
        return(z)
    elif re.match(two_three, item):
        x = re.split(r'[,&()]\s*', item)
        z = [x[1], x[2], x[5], x[6], x[7]]
        return(z)
    elif re.match(four_and_one, item):
        x = re.split(r'[,&()]\s*', item)
        z = [x[1], x[2], x[3], x[4], x[6]]
        return(z)
    elif re.match(fivesome, item):
        x = re.split(r'[,&()]\s*', item)
        z = [x[1], x[2], x[3], x[4], x[5]]
        return(z)
    else:
        x = re.split(r'[,&()]\s*', item)
        z = [x[0], x[1], x[2], x[3], x[4]]
        return(z)

In [454]:
tag10_matches = r[r['matchtype'] == 'tag10']

tag10_matches['win'] = [tag_splitter(i)[0] for i in tag10_matches['result']]
tag10_matches['lose'] = [tag_splitter(i)[1] for i in tag10_matches['result']]
tag10_matches['time'] = [tag_splitter(i)[2] if len(tag_splitter(i)) == 3 else '(Unk.)' for i in tag10_matches['result']]

tag10_matches.win = [i.rstrip().lstrip() for i in tag10_matches['win'].str.replace(r"\(w\/.*?\)","")]
tag10_matches.lose = [i.rstrip().lstrip() for i in tag10_matches['lose'].str.replace(r"\(w\/.*?\)","")]

tag10_matches.win = [tag10_parser(i) for i in tag10_matches.win]
tag10_matches.lose = [tag10_parser(i) for i in tag10_matches.lose]

tag10_matches['win1'] = [i[0] for i in tag10_matches.win]
tag10_matches['win2'] = [i[1] for i in tag10_matches.win]
tag10_matches['win3'] = [i[2] for i in tag10_matches.win]
tag10_matches['win4'] = [i[3] for i in tag10_matches.win]
tag10_matches['win5'] = [i[4] for i in tag10_matches.win]

tag10_matches['lose1'] = [i[0] for i in tag10_matches.lose]
tag10_matches['lose2'] = [i[1] for i in tag10_matches.lose]
tag10_matches['lose3'] = [i[2] for i in tag10_matches.lose]
tag10_matches['lose4'] = [i[3] for i in tag10_matches.lose]
tag10_matches['lose5'] = [i[4] for i in tag10_matches.lose]

tag10_matches = tag10_matches[['Event', 'Date', 'result', 'matchtype', 'win1', 'win2', 'win3', 'win4', 'win5', 'lose1', 'lose2', 'lose3', 'lose4', 'lose5', 'time']]

tag10_matches

,Event,Date,result,matchtype,win1,win2,win3,win4,win5,lose1,lose2,lose3,lose4,lose5,time
18,AEW Dynamite #84 - Blood & Guts,05.05.2021,"The Pinnacle (Cash Wheeler, Dax Harwood, MJF, ...",tag10,Cash Wheeler,Dax Harwood,MJF,Shawn Spears,Wardlow,Chris Jericho,Jake Hager,Ortiz,Sammy Guevara,Santana,(17:00)
90,AEW The House Always Wins,09.04.2021,"Death Triangle (PAC, Penta El Zero Miedo & Rey...",tag10,PAC,Penta El Zero Miedo,Rey Fenix,Matt Sydal,Mike Sydal,Matt Jackson,Nick Jackson,Kenny Omega,Konosuke Takeshita,Michael Nakazawa,(12:25)
228,AEW Dynamite #77 - St. Patrick's Day Slam,17.03.2021,"Matt Hardy, Private Party (Isiah Kassidy & Mar...",tag10,Matt Hardy,Isiah Kassidy,Marq Quen,The Blade,The Butcher,Bear Boulder,Bear Bronson,Jungle Boy,Luchasaurus,Marko Stunt,(8:47)
489,AEW Dark #70,12.01.2021,"The Dark Order (Colt Cabana, Evil Uno, Five, S...",tag10,Colt Cabana,Evil Uno,Five,Stu Grayson,Ten,Adam Priest,Danny Limelight,Shawn Dean,Vary Morales,Zack Clayton,(8:19)
934,AEW Dynamite #43,29.07.2020,"Best Friends (Chuck Taylor & Trent), The Juras...",tag10,Chuck Taylor,Trent,Jungle Boy,Luchasaurus,Orange Cassidy,Chris Jericho,Jake Hager,Ortiz,Sammy Guevara,Santana,(12:06)
1083,AEW Double Or Nothing 2020,23.05.2020,"The Elite (Adam Page, Kenny Omega, Matt Jackso...",tag10,Adam Page,Kenny Omega,Matt Jackson,Nick Jackson,Matt Hardy,Chris Jericho,Jake Hager,Ortiz,Sammy Guevara,Santana,(33:59)


#### parsing tag12s

In [455]:
def tag12_parser(item):
    four_and_two = '.*? \(.*\) & .* \(.*\)'
    five_and_one = '.*? \(.*\) & .*'
    two_two_two =  '.*? \(.*\), .*? \(.*\) & .*? \(.*\)'
    sixsome = '.*? \(.*\)'
    
    if re.match(two_two_two, item):
        x = re.split(r'[,&()]\s*', item)
        z = [x[1], x[2], x[5], x[6], x[9], x[10]]
        return(z)
    elif re.match(four_and_two, item):
        x = re.split(r'[,&()]\s*', item)
        z = [x[1], x[2], x[3], x[4], x[7], x[8]]
        return(z)
    elif re.match(five_and_one, item):
        x = re.split(r'[,&()]\s*', item)
        z = [x[1], x[2], x[3], x[4], x[5], x[7]]
        return(z)
    elif re.match(sixsome, item):
        x = re.split(r'[,&()]\s*', item)
        z = [x[1], x[2], x[3], x[4], x[5], x[6]]
        return(z)
    else:
        x = re.split(r'[,&()]\s*', item)
        z = [x[0], x[1], x[2], x[3], x[4]]
        return(z)

In [456]:
tag12_matches = r[r['matchtype'] == 'tag12']

tag12_matches['win'] = [tag_splitter(i)[0] for i in tag12_matches['result']]
tag12_matches['lose'] = [tag_splitter(i)[1] for i in tag12_matches['result']]
tag12_matches['time'] = [tag_splitter(i)[2] if len(tag_splitter(i)) == 3 else '(Unk.)' for i in tag12_matches['result']]

tag12_matches.win = [i.rstrip().lstrip() for i in tag12_matches['win'].str.replace(r"\(w\/.*?\)","")]
tag12_matches.lose = [i.rstrip().lstrip() for i in tag12_matches['lose'].str.replace(r"\(w\/.*?\)","")]

tag12_matches.win = [tag12_parser(i) for i in tag12_matches.win]
tag12_matches.lose = [tag12_parser(i) for i in tag12_matches.lose]

tag12_matches['win1'] = [i[0] for i in tag12_matches.win]
tag12_matches['win2'] = [i[1] for i in tag12_matches.win]
tag12_matches['win3'] = [i[2] for i in tag12_matches.win]
tag12_matches['win4'] = [i[3] for i in tag12_matches.win]
tag12_matches['win5'] = [i[4] for i in tag12_matches.win]
tag12_matches['win6'] = [i[5] for i in tag12_matches.win]

tag12_matches['lose1'] = [i[0] for i in tag12_matches.lose]
tag12_matches['lose2'] = [i[1] for i in tag12_matches.lose]
tag12_matches['lose3'] = [i[2] for i in tag12_matches.lose]
tag12_matches['lose4'] = [i[3] for i in tag12_matches.lose]
tag12_matches['lose5'] = [i[4] for i in tag12_matches.lose]
tag12_matches['lose6'] = [i[5] for i in tag12_matches.lose]

tag12_matches = tag12_matches[['Event', 'Date', 'result', 'matchtype', 'win1', 'win2', 'win3', 'win4', 'win5', 'win6', 'lose1', 'lose2', 'lose3', 'lose4', 'lose5', 'lose6', 'time']]


tag12_matches

,Event,Date,result,matchtype,win1,win2,win3,win4,win5,win6,lose1,lose2,lose3,lose4,lose5,lose6,time
554,AEW Dynamite #64,16.12.2020,"The Inner Circle (Chris Jericho, Jake Hager, M...",tag12,Chris Jericho,Jake Hager,MJF,Ortiz,Sammy Guevara,Santana,Chuck Taylor,Trent,Brian Pillman Jr.,Griff Garrison,Darius Martin,Daunte Martin,(14:19)
920,AEW Dynamite #44,05.08.2020,"The Dark Order (Evil Uno, Five, Mr. Brodie Lee...",tag12,Evil Uno,Five,Mr. Brodie Lee,Nine,Stu Grayson,Colt Cabana,Adam Page,Kenny Omega,Matt Jackson,Nick Jackson,Cash Wheeler,Dax Harwood,(17:19)


#### parsing fourways

In [457]:
fourway_matches = r[r['matchtype'] == 'fourway']


fourway_matches['time'] = [singles_splitter(i)[2] if len(singles_splitter(i)) == 3 else '(Unk.)' for i in fourway_matches['result']]
fourway_matches.result = [i.rstrip().lstrip() for i in fourway_matches['result'].str.replace(r"\(w\/.*?\)","")]

fourway_matches['win1'] = [singles_splitter(i)[0] for i in fourway_matches['result']]
fourway_matches['lose'] = [singles_splitter(i)[1] for i in fourway_matches['result']]
fourway_matches[['lose1', 'lose2', 'lose3']] = [(multiway_splitter(i)[0], multiway_splitter(i)[1], multiway_splitter(i)[2]) for i in fourway_matches.lose]


fourway_matches = fourway_matches[['Event', 'Date', 'result', 'matchtype', 'win1', 'lose1', 'lose2', 'lose3', 'time']]


fourway_matches

,Event,Date,result,matchtype,win1,lose1,lose2,lose3,time
516,AEW Dark #69,05.01.2021,Frankie Kazarian defeats Angelico and Darius...,fourway,Frankie Kazarian,Angelico,Darius Martin,Griff Garrison,(7:43)
759,AEW Dark #56,13.10.2020,Jungle Boy defeats Evil Uno and Frankie Kazari...,fourway,Jungle Boy,Evil Uno,Frankie Kazarian,The Blade,(8:29)
1101,AEW Dynamite #32,13.05.2020,Hikaru Shida defeats Dr. Britt Baker DMD and K...,fourway,Hikaru Shida,Dr. Britt Baker DMD,Kris Statlander,Penelope Ford,(10:12)
1184,AEW Dynamite #24,18.03.2020,Hikaru Shida defeats Kris Statlander and Penel...,fourway,Hikaru Shida,Kris Statlander,Penelope Ford,Riho,(5:49)
1223,AEW Dynamite #21,26.02.2020,Hikaru Shida defeats Big Swole and Shanna and ...,fourway,Hikaru Shida,Big Swole,Shanna,Yuka Sakazaki,(9:13)
1289,AEW Dynamite #13 - Homecoming Edition,01.01.2020,Riho defeats Dr. Britt Baker DMD and Hikaru S...,fourway,Riho,Dr. Britt Baker DMD,Hikaru Shida,Nyla Rose,(9:49)
1387,AEW Dark #4,29.10.2019,Emi Sakura defeats Allie and Penelope Ford and...,fourway,Emi Sakura,Allie,Penelope Ford,Sadie Gibbs,(11:54)
1436,AEW Fyter Fest,29.06.2019,Adam Page defeats Jimmy Havoc and Jungle Boy ...,fourway,Adam Page,Jimmy Havoc,Jungle Boy,Maxwell Jacob Friedman,(10:51)
1443,AEW Double Or Nothing 2019,25.05.2019,Dr. Britt Baker DMD defeats Awesome Kong and ...,fourway,Dr. Britt Baker DMD,Awesome Kong,Kylie Rae,Nyla Rose,(11:08)


#### parsing threeways

In [458]:
threeway_matches = r[r['matchtype'] == 'threeway']


threeway_matches['time'] = [singles_splitter(i)[2] if len(singles_splitter(i)) == 3 else '(Unk.)' for i in threeway_matches['result']]
threeway_matches.result = [i.rstrip().lstrip() for i in threeway_matches['result'].str.replace(r"\(w\/.*?\)","")]

threeway_matches['win1'] = [singles_splitter(i)[0] for i in threeway_matches['result']]
threeway_matches['lose'] = [singles_splitter(i)[1] for i in threeway_matches['result']]
threeway_matches[['lose1', 'lose2']] = [(multiway_splitter(i)[0], multiway_splitter(i)[1]) for i in threeway_matches.lose]


threeway_matches = threeway_matches[['Event', 'Date', 'result', 'matchtype', 'win1', 'lose1', 'lose2', 'time']]


threeway_matches

,Event,Date,result,matchtype,win1,lose1,lose2,time
1173,AEW @The BTE Compound,13.04.2020,Scorpio Sky defeats Brandon Cutler and Peter A...,threeway,Scorpio Sky,Brandon Cutler,Peter Avalon,(Unk.)
1326,AEW Dark #10,10.12.2019,Scorpio Sky defeats Jimmy Havoc and Peter Aval...,threeway,Scorpio Sky,Jimmy Havoc,Peter Avalon,(6:28)
1350,AEW Dynamite #7,13.11.2019,Darby Allin defeats Peter Avalon and Shawn Sp...,threeway,Darby Allin,Peter Avalon,Shawn Spears,(3:44)
1388,AEW Dark #4,29.10.2019,Darby Allin defeats Jack Evans and Jimmy Havoc...,threeway,Darby Allin,Jack Evans,Jimmy Havoc,(13:12)
1403,AEW Dark #2,15.10.2019,Kip Sabian defeats Peter Avalon and Sonny Kis...,threeway,Kip Sabian,Peter Avalon,Sonny Kiss,(6:06)
1417,AEW All Out 2019,31.08.2019,Jimmy Havoc defeats Darby Allin and Joey Janel...,threeway,Jimmy Havoc,Darby Allin,Joey Janela,(14:57)
1435,AEW Fyter Fest,29.06.2019,Riho defeats Nyla Rose and Yuka Sakazaki (13:02),threeway,Riho,Nyla Rose,Yuka Sakazaki,(13:02)


#### parsing fourwaytags

In [459]:
fourwaytag_matches = r[r['matchtype'] == 'fourwaytag']
fourwaytag_matches['time'] = [tag_splitter(i)[2] if len(tag_splitter(i)) == 3 else '(Unk.)' for i in fourwaytag_matches['result']]
fourwaytag_matches.result = [i.rstrip().lstrip() for i in fourwaytag_matches['result'].str.replace(r"\(w\/.*?\)","")]

fourwaytag_matches['win_team'] = [tag_splitter(i)[0] for i in fourwaytag_matches['result']]
fourwaytag_matches['lose_teams'] = [tag_splitter(i)[1] for i in fourwaytag_matches['result']]
fourwaytag_matches[['loseteam1', 'loseteam2', 'loseteam3']] = [(multiway_splitter(i)[0], multiway_splitter(i)[1], multiway_splitter(i)[2]) for i in fourwaytag_matches.lose_teams]

fourwaytag_matches['win_team'] = [tag_team_parser(i) for i in fourwaytag_matches.win_team]
fourwaytag_matches['loseteam1'] = [tag_team_parser(i) for i in fourwaytag_matches.loseteam1]
fourwaytag_matches['loseteam2'] = [tag_team_parser(i) for i in fourwaytag_matches.loseteam2]
fourwaytag_matches['loseteam3'] = [tag_team_parser(i) for i in fourwaytag_matches.loseteam3]

fourwaytag_matches[['win1', 'win2']] = fourwaytag_matches.win_team.str.split(" & ",expand=True)
fourwaytag_matches[['lose1', 'lose2']] = fourwaytag_matches.loseteam1.str.split(" & ",expand=True) 
fourwaytag_matches[['lose3', 'lose4']] = fourwaytag_matches.loseteam2.str.split(" & ",expand=True) 
fourwaytag_matches[['lose5', 'lose6']] = fourwaytag_matches.loseteam3.str.split(" & ",expand=True) 

fourwaytag_matches = fourwaytag_matches[['Event', 'Date', 'result', 'matchtype', 'win1', 'win2', 'lose1', 'lose2', 'lose3', 'lose4', 'lose5', 'lose6', 'time']]


fourwaytag_matches

,Event,Date,result,matchtype,win1,win2,lose1,lose2,lose3,lose4,lose5,lose6,time
17,AEW Dynamite #84 - Blood & Guts,05.05.2021,SCU (Christopher Daniels & Frankie Kazarian) d...,fourwaytag,Christopher Daniels,Frankie Kazarian,Brian Pillman Jr.,Griff Garrison,Jungle Boy,Luchasaurus,Anthony Bowens,Max Caster,(9:12)
718,AEW Dynamite #56,21.10.2020,The Young Bucks (Matt Jackson & Nick Jackson) ...,fourwaytag,Matt Jackson,Nick Jackson,Isiah Kassidy,Marq Quen,The Blade,The Butcher,Alex Reynolds,John Silver,(13:29)
869,AEW Dynamite #47 - Thursday Night Dynamite,27.08.2020,FTR (Cash Wheeler & Dax Harwood) defeat Best ...,fourwaytag,Cash Wheeler,Dax Harwood,Chuck Taylor,Trent,Dustin Rhodes,QT Marshall,Matt Jackson,Nick Jackson,(5:25)
1269,AEW Dynamite #15 - Bash At The Beach,15.01.2020,Adam Page & Kenny Omega defeat Best Friends (C...,fourwaytag,Adam Page,Kenny Omega,Chuck Taylor,Trent,Ortiz,Santana,Matt Jackson,Nick Jackson,(16:35)
1308,AEW Dark #12,24.12.2019,Proud-N-Powerful (Ortiz & Santana) defeat Best...,fourwaytag,Ortiz,Santana,Chuck Taylor,Trent,Isiah Kassidy,Marq Quen,Angelico,Jack Evans,(16:08)


#### parsing threewaytags

In [460]:
threewaytag_matches = r[r['matchtype'] == 'threewaytag']
threewaytag_matches['time'] = [tag_splitter(i)[2] if len(tag_splitter(i)) == 3 else '(Unk.)' for i in threewaytag_matches['result']]
threewaytag_matches.result = [i.rstrip().lstrip() for i in threewaytag_matches['result'].str.replace(r"\(w\/.*?\)","")]

threewaytag_matches['win_team'] = [tag_splitter(i)[0] for i in threewaytag_matches['result']]
threewaytag_matches['lose_teams'] = [tag_splitter(i)[1] for i in threewaytag_matches['result']]
threewaytag_matches[['loseteam1', 'loseteam2']] = [(multiway_splitter(i)[0], multiway_splitter(i)[1]) for i in threewaytag_matches.lose_teams]

threewaytag_matches['win_team'] = [tag_team_parser(i) for i in threewaytag_matches.win_team]
threewaytag_matches['loseteam1'] = [tag_team_parser(i) for i in threewaytag_matches.loseteam1]
threewaytag_matches['loseteam2'] = [tag_team_parser(i) for i in threewaytag_matches.loseteam2]

threewaytag_matches[['win1', 'win2']] = threewaytag_matches.win_team.str.split(" & ",expand=True)
threewaytag_matches[['lose1', 'lose2']] = threewaytag_matches.loseteam1.str.split(" & ",expand=True) 
threewaytag_matches[['lose3', 'lose4']] = threewaytag_matches.loseteam2.str.split(" & ",expand=True) 

threewaytag_matches = threewaytag_matches[['Event', 'Date', 'result', 'matchtype', 'win1', 'win2', 'lose1', 'lose2', 'lose3', 'lose4', 'time']]

threewaytag_matches

,Event,Date,result,matchtype,win1,win2,lose1,lose2,lose3,lose4,time
380,AEW Dark #76,23.02.2021,SCU (Christopher Daniels & Frankie Kazarian) d...,threewaytag,Christopher Daniels,Frankie Kazarian,Angelico,Jack Evans,Matt Sydal,Mike Sydal,(11:38)
444,AEW Dynamite #69,20.01.2021,The Inner Circle (Chris Jericho & MJF) defeat...,threewaytag,Chris Jericho,MJF,Jake Hager,Sammy Guevara,Ortiz,Santana,(12:38)
621,AEW Dark #64,01.12.2020,The Jurassic Express (Jungle Boy & Luchasaurus...,threewaytag,Jungle Boy,Luchasaurus,Five,Ten,Joey Janela,Sonny Kiss,(9:35)
643,AEW Dark #63,24.11.2020,The Dark Order (Evil Uno & Stu Grayson) defeat...,threewaytag,Evil Uno,Stu Grayson,Luther,Serpentico,Joey Janela,Sonny Kiss,(9:02)
969,AEW Dark #44,28.07.2020,The Inner Circle (Ortiz & Santana) defeat SCU ...,threewaytag,Ortiz,Santana,Christopher Daniels,Frankie Kazarian,Isiah Kassidy,Marq Quen,(11:01)
1360,AEW Full Gear 2019,09.11.2019,SCU (Frankie Kazarian & Scorpio Sky) defeat T...,threewaytag,Frankie Kazarian,Scorpio Sky,Pentagon Jr.,Rey Fenix,Isiah Kassidy,Marq Quen,(13:07)
1427,AEW Fight For The Fallen,13.07.2019,The Dark Order (Evil Uno & Stu Grayson) defeat...,threewaytag,Evil Uno,Stu Grayson,Jungle Boy,Luchasaurus,Angelico,Jack Evans,(15:07)
1439,AEW Fyter Fest - The Buy In,29.06.2019,Best Friends (Chuck Taylor & Trent) defeat SCU...,threewaytag,Chuck Taylor,Trent,Frankie Kazarian,Scorpio Sky,Isiah Kassidy,Marq Quen,(15:56)


#### parsing handicaps

In [461]:
handicap3on1_matches = r[r['matchtype'] == 'handicap3on1']

handicap3on1_matches['time'] = [singles_splitter(i)[2] if len(singles_splitter(i)) == 3 else '(Unk.)' for i in handicap3on1_matches['result']]
handicap3on1_matches.result = [i.rstrip().lstrip() for i in handicap3on1_matches['result'].str.replace(r"\(w\/.*?\)","")]

handicap3on1_matches['win1'] = [singles_splitter(i)[0] for i in handicap3on1_matches['result']]
handicap3on1_matches['lose_teams'] = [singles_splitter(i)[1] for i in handicap3on1_matches['result']]
handicap3on1_matches[['lose1', 'lose2', 'lose3']] = [re.split(r'[,&()]\s*', i) for i in handicap3on1_matches.lose_teams]

handicap3on1_matches = handicap3on1_matches[['Event', 'Date', 'result', 'matchtype', 'win1', 'lose1', 'lose2', 'lose3']]


handicap3on1_matches

,Event,Date,result,matchtype,win1,lose1,lose2,lose3
872,AEW Dynamite #47 - Thursday Night Dynamite,27.08.2020,"Big Swole defeats Dr. Britt Baker DMD, Penelop...",handicap3on1,Big Swole,Dr. Britt Baker DMD,Penelope Ford,Reba


In [462]:
handicap2on1_matches = r[r['matchtype'] == 'handicap2on1']

handicap2on1_matches['time'] = [singles_splitter(i)[2] if len(singles_splitter(i)) == 3 else '(Unk.)' for i in handicap2on1_matches['result']]
handicap2on1_matches.result = [i.rstrip().lstrip() for i in handicap2on1_matches['result'].str.replace(r"\(w\/.*?\)","")]

handicap2on1_matches['win'] = [singles_splitter(i)[0] for i in handicap2on1_matches['result']]
handicap2on1_matches['win_teams'] = [tag_splitter(i)[0] for i in handicap2on1_matches['result']]
handicap2on1_matches['winteam1'] = [tag_team_parser(i) for i in handicap2on1_matches.win_teams]
handicap2on1_matches[['win1', 'win2']] = handicap2on1_matches.winteam1.str.split(" & ",expand=True) 
handicap2on1_matches['lose_teams'] = [tag_splitter(i)[1] for i in handicap2on1_matches['result']]
handicap2on1_matches['loseteam1'] = [tag_team_parser(i) for i in handicap2on1_matches.lose_teams]
handicap2on1_matches[['lose1', 'lose2']] = handicap2on1_matches.loseteam1.str.split(" & ",expand=True) 
handicap2on1_matches['lose1'] = [i[2:] if i[:2] == 's ' else i for i in handicap2on1_matches.lose1]

handicap2on1_matches = handicap2on1_matches[['Event', 'Date', 'result', 'matchtype', 'win1', 'win2', 'lose1', 'lose2', 'time']]

handicap2on1_matches

,Event,Date,result,matchtype,win1,win2,lose1,lose2,time
917,AEW Dark #47,18.08.2020,Lance Archer defeats Jessy Sorensen & Jon Cru...,handicap2on1,Lance Archer,None,Jessy Sorensen,Jon Cruz,(2:42)
986,AEW Dynamite #40 - Fyter Fest 2020 - Tag 2,08.07.2020,Nyla Rose defeats Kenzie Paige & KiLynn King (...,handicap2on1,Nyla Rose,None,Kenzie Paige,KiLynn King,(2:21)
1206,AEW Dynamite #22,04.03.2020,The Inner Circle (Chris Jericho & Sammy Guevar...,handicap2on1,Chris Jericho,Sammy Guevara,Darby Allin,None,(Unk.)


#### other parses

In [463]:
nineway_matches = r[r['matchtype'] == 'nineway']


nineway_matches['time'] = [singles_splitter(i)[2] if len(singles_splitter(i)) == 3 else '(Unk.)' for i in nineway_matches['result']]
nineway_matches.result = [i.rstrip().lstrip() for i in nineway_matches['result'].str.replace(r"\(w\/.*?\)","")]

nineway_matches['win1'] = [singles_splitter(i)[0] for i in nineway_matches['result']]
nineway_matches['lose'] = [singles_splitter(i)[1] for i in nineway_matches['result']]
nineway_matches[['lose1', 'lose2', 'lose3', 'lose4', 'lose5', 'lose6', 'lose7', 'lose8']] = \
[(multiway_splitter(i)[0], multiway_splitter(i)[1], multiway_splitter(i)[2], multiway_splitter(i)[3], \
  multiway_splitter(i)[4], multiway_splitter(i)[5], multiway_splitter(i)[6], multiway_splitter(i)[7], \
 ) for i in nineway_matches.lose]


nineway_matches = nineway_matches[['Event', 'Date', 'result', 'matchtype', 'win1', 'lose1', 'lose2', \
                                   'lose3', 'lose4', 'lose5', 'lose6', 'lose7', 'lose8', 'time']]


nineway_matches

,Event,Date,result,matchtype,win1,lose1,lose2,lose3,lose4,lose5,lose6,lose7,lose8,time
1076,AEW Double Or Nothing 2020,23.05.2020,Brian Cage defeats Colt Cabana and Darby Alli...,nineway,Brian Cage,Colt Cabana,Darby Allin,Frankie Kazarian,Kip Sabian,Joey Janela,Luchasaurus,Orange Cassidy,Scorpio Sky,(27:28)


In [464]:
sixway_matches = r[r['matchtype'] == 'sixway']


sixway_matches['time'] = [singles_splitter(i)[2] if len(singles_splitter(i)) == 3 else '(Unk.)' for i in sixway_matches['result']]
sixway_matches.result = [i.rstrip().lstrip() for i in sixway_matches['result'].str.replace(r"\(w\/.*?\)","")]

sixway_matches['win1'] = [singles_splitter(i)[0] for i in sixway_matches['result']]
sixway_matches['lose'] = [singles_splitter(i)[1] for i in sixway_matches['result']]
sixway_matches[['lose1', 'lose2', 'lose3', 'lose4', 'lose5']] = \
[(multiway_splitter(i)[0], multiway_splitter(i)[1], multiway_splitter(i)[2], multiway_splitter(i)[3], \
  multiway_splitter(i)[4]) for i in sixway_matches.lose]


sixway_matches = sixway_matches[['Event', 'Date', 'result', 'matchtype', 'win1', 'lose1', 'lose2', \
                                   'lose3', 'lose4', 'lose5', 'time']]


sixway_matches

,Event,Date,result,matchtype,win1,lose1,lose2,lose3,lose4,lose5,time
295,AEW Revolution 2021,07.03.2021,Scorpio Sky defeats Cody Rhodes and Ethan Pag...,sixway,Scorpio Sky,Cody Rhodes,Ethan Page,Lance Archer,Max Caster,Penta El Zero Miedo,(23:07)


### merge

In [465]:
data_frames = [singles_matches, tag_matches, tag6_matches, tag8_matches, tag10_matches, tag12_matches, fourway_matches,\
               threeway_matches, threewaytag_matches, fourwaytag_matches, handicap3on1_matches, handicap2on1_matches,\
               sixway_matches, nineway_matches]

merged_df = pd.concat(data_frames)
merged_df[merged_df.columns] = merged_df.apply(lambda x: x.str.strip())
merged_df['win1'] = [re.sub('\(vakant\)', '', x) for x in merged_df.win1]

merged_df = merged_df[['Event', 'Date', 'result', 'matchtype', 'time',\
                     'win1', 'win2', 'win3', 'win4', 'win5', 'win6',\
                     'lose1', 'lose2', 'lose3', 'lose4', 'lose5', 'lose6', 'lose7', 'lose8']]

draws = pd.read_csv('all_draws.csv')
merged_df = pd.concat([merged_df, draws])

merged_df.to_csv('merged_' + today + '.csv')

In [466]:
merged_df

,Event,Date,result,matchtype,time,win1,win2,win3,win4,win5,win6,lose1,lose2,lose3,lose4,lose5,lose6,lose7,lose8
0,AEW Dark: Elevation #9,10.05.2021,Lee Johnson defeats David Ali,singles,(Unk.),Lee Johnson,NaN,NaN,NaN,NaN,NaN,David Ali,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AEW Dark: Elevation #9,10.05.2021,Eddie Kingston defeats VSK,singles,(Unk.),Eddie Kingston,NaN,NaN,NaN,NaN,NaN,VSK,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AEW Dark: Elevation #9,10.05.2021,Thunder Rosa defeats Renee Michelle,singles,(Unk.),Thunder Rosa,NaN,NaN,NaN,NaN,NaN,Renee Michelle,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AEW Dark: Elevation #9,10.05.2021,"Chuck Taylor (w/Kris Statlander, Orange Cassid...",singles,(Unk.),Chuck Taylor,NaN,NaN,NaN,NaN,NaN,Vary Morales,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,AEW Dark: Elevation #9,10.05.2021,Leyla Hirsch defeats Dani Jordyn,singles,(Unk.),Leyla Hirsch,NaN,NaN,NaN,NaN,NaN,Dani Jordyn,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,AEW Dark #56,13.10.2020,Brandon Cutler vs. Peter Avalon - Double DQ,draw,(7:27),Brandon Cutler,NaN,NaN,NaN,NaN,NaN,Peter Avalon,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AEW Dark #52,15.09.2020,Brandon Cutler vs. Peter Avalon - Double Count...,draw,(6:45),Brandon Cutler,NaN,NaN,NaN,NaN,NaN,Peter Avalon,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AEW Dynamite #4,23.10.2019,Jon Moxley vs. PAC - Time Limit Draw (12:10),draw,(12:10),Jon Moxley,NaN,NaN,NaN,NaN,NaN,PAC,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AEW Dynamite #55 - Anniversary Show,14.10.2020,Cody (w/Arn Anderson & Brandi Rhodes) (c) vs. ...,draw,(20:00),Cody,NaN,NaN,NaN,NaN,NaN,Orange Cassidy,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Trueskill

In [467]:
alias_dict = pd.read_excel('alias_dict.xlsx')
aliases = list(alias_dict.alias)

def alias_collapser(name):
    y = list(alias_dict[alias_dict['alias'] == name]['name'])
    return y[0]

In [468]:
all_matches = pd.read_csv('merged_' + today + '.csv')
all_matches = all_matches.drop(columns=['Unnamed: 0'])
all_matches['Date'] = pd.to_datetime(all_matches.Date, format='%d.%m.%Y')
all_matches = all_matches.sort_values('Date')


all_matches.tail()

,Event,Date,result,matchtype,time,win1,win2,win3,win4,win5,win6,lose1,lose2,lose3,lose4,lose5,lose6,lose7,lose8
10,AEW Dark: Elevation #9,2021-05-10,Jon Moxley defeats Danny Limelight,singles,(Unk.),Jon Moxley,NaN,NaN,NaN,NaN,NaN,Danny Limelight,NaN,NaN,NaN,NaN,NaN,NaN,NaN
844,AEW Dark: Elevation #9,2021-05-10,Ethan Page & Scorpio Sky defeat Baron Black & ...,tag,(Unk.),Ethan Page,Scorpio Sky,NaN,NaN,NaN,NaN,Baron Black,Fuego Del Sol,NaN,NaN,NaN,NaN,NaN,NaN
845,AEW Dark: Elevation #9,2021-05-10,The Hardy Family Office (Marq Quen & Matt Hard...,tag,(Unk.),Marq Quen,Matt Hardy,NaN,NaN,NaN,NaN,Dean Alexander,Dillon McQueen,NaN,NaN,NaN,NaN,NaN,NaN
4,AEW Dark: Elevation #9,2021-05-10,Leyla Hirsch defeats Dani Jordyn,singles,(Unk.),Leyla Hirsch,NaN,NaN,NaN,NaN,NaN,Dani Jordyn,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,AEW Dark: Elevation #9,2021-05-10,Lee Johnson defeats David Ali,singles,(Unk.),Lee Johnson,NaN,NaN,NaN,NaN,NaN,David Ali,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [469]:
x = Rating(25, 8)
x.mu

Rating()

trueskill.Rating(mu=25.000, sigma=8.333)

In [470]:
def create_player_frame():
    '''
    initializes dataframe
    '''
    player_frame = pd.DataFrame(columns=['name', 'mu', 'sigma'])
    return player_frame

def create_macro_player_frame():
    '''
    initializes by-date dataframe
    '''
    player_frame = pd.DataFrame(columns=['name', 'mu', 'sigma','Date', 'Result'])
    return player_frame


def player_add(name, player_frame):
    '''
    adds player to created player frame
    '''
    x = Rating()
    player_frame  = player_frame.append({'name': name, \
                                         'mu': x.mu, 'sigma': x.sigma}, ignore_index=True)
    
    return player_frame

def player_array(name, player_frame):
    '''
    returns mu, sigma (in that order) of player's trueskill
    '''
    
    mu = player_frame[player_frame['name'] == name]['mu']
    sigma = player_frame[player_frame['name'] == name]['sigma']
    
    mu = list(mu)
    sigma = list(sigma)
    
    mu1 = mu[0]
    sigma1 = sigma[0]
    
    return [mu1, sigma1]

def victory_1v1(winner, loser, player_frame):
    '''
    updates player_frame after a 1v1 matchup that is not a tie
    '''
    
    winner_mu, winner_sigma = player_array(winner, player_frame)
    loser_mu, loser_sigma = player_array(loser, player_frame)
    
    win_rate = Rating(winner_mu, winner_sigma)
    lose_rate = Rating(loser_mu, loser_sigma)
    
    result = rate([(win_rate,), (lose_rate,)], ranks=[0,1])
    (win,), (lose,), = result
    
    player_frame = player_frame[player_frame['name'] != winner]
    player_frame = player_frame[player_frame['name'] != loser]
    
    player_frame  = player_frame.append({'name': loser, \
                                         'mu': lose.mu, 'sigma': lose.sigma}, ignore_index=True)
    player_frame  = player_frame.append({'name': winner, \
                                         'mu': win.mu, 'sigma': win.sigma}, ignore_index=True)
    
    return player_frame

def get_rating(name, player_frame):
    '''
    '''
    mu, sigma = player_array(name, player_frame)
    rating = Rating(mu, sigma)
    return rating

def victory_singles(result_list, player_frame):
    '''
    updates player_frame after any singles matchup that is not a tie (regardless of participant count)
    '''
    rate_pt1 = []
    for item in result_list:
        x = (get_rating(item, player_frame),)
        rate_pt1.append(x)
        
    y = len(result_list)
    rate_pt2 = [0]
    for i in [1] * (y-1):
        rate_pt2.append(1)
        
    result = rate(rate_pt1, ranks=rate_pt2)
    
    player_frame_old = player_frame
    
    for item in result_list:
        player_frame = player_frame[player_frame['name'] != item]
    
    for i in range(0,y):
        player_frame = player_frame.append({'name': result_list[i], \
                                         'mu': result[i][0].mu, 'sigma': result[i][0].sigma}, ignore_index=True)
        
    #added 4.4.2021: should prevent losers in multiway matches from upping their rankings because of competition
    for item in result_list[1:]:
        if get_rating(item, player_frame).mu > get_rating(item, player_frame_old).mu:
            player_frame.loc[(player_frame.name == item),'mu'] = get_rating(item, player_frame_old).mu
    return player_frame

def victory_draw(result_list, player_frame):
    '''
    updates player_frame after any singles matchup that is a tie (regardless of participant count)
    '''
    rate_pt1 = []
    for item in result_list:
        x = (get_rating(item, player_frame),)
        rate_pt1.append(x)
        
    y = len(result_list)
    rate_pt2 = [0]
    for i in [1] * (y-1):
        rate_pt2.append(0)
        
    result = rate(rate_pt1, ranks=[0,0])
    
    player_frame_old = player_frame
    
    for item in result_list:
        player_frame = player_frame[player_frame['name'] != item]
    
    for i in range(0,y):
        player_frame = player_frame.append({'name': result_list[i], \
                                         'mu': result[i][0].mu, 'sigma': result[i][0].sigma}, ignore_index=True)
        
    #added 4.4.2021: should prevent losers in multiway matches from upping their rankings because of competition
    for item in result_list[0:]:
        if get_rating(item, player_frame).mu > get_rating(item, player_frame_old).mu:
            player_frame.loc[(player_frame.name == item),'mu'] = get_rating(item, player_frame_old).mu
    return player_frame

def victory_tag(result_list, player_frame):
    '''
    updates player_frame after a tag matchup that is not a tie
    '''
    
    x1 = Rating(get_rating(result_list[0], player_frame),)
    x2 = Rating(get_rating(result_list[1], player_frame),)

    y1 = Rating(get_rating(result_list[2], player_frame),)
    y2 = Rating(get_rating(result_list[3], player_frame),)

    rate_pt2 = [0, 1]
    
    result = rate([(x1,x2), (y1,y2)], ranks=[0,1])

    result = [item for sublist in result for item in sublist]

    y = len(result_list)
    
    player_frame_old = player_frame

    for item in result_list:
        player_frame = player_frame[player_frame['name'] != item]
    
    for i in range(0,y):
        player_frame  = player_frame.append({'name': result_list[i], \
                                         'mu': result[i].mu, 'sigma': result[i].sigma}, ignore_index=True)
    
    #added 4.4.2021: should prevent losers in multiway matches from upping their rankings because of competition
    for item in result_list[2:]:
        if get_rating(item, player_frame).mu > get_rating(item, player_frame_old).mu:
            player_frame.loc[(player_frame.name == item),'mu'] = get_rating(item, player_frame_old).mu

    return player_frame

def victory_threewaytag(result_list, player_frame):
    '''
    updates player_frame after a tag matchup that is not a tie
    '''
    
    x1 = Rating(get_rating(result_list[0], player_frame),)
    x2 = Rating(get_rating(result_list[1], player_frame),)

    y1 = Rating(get_rating(result_list[2], player_frame),)
    y2 = Rating(get_rating(result_list[3], player_frame),)
    
    z1 = Rating(get_rating(result_list[4], player_frame),)
    z2 = Rating(get_rating(result_list[5], player_frame),)

    rate_pt2 = [0, 1, 1]
    
    result = rate([(x1,x2), (y1,y2), (z1, z2)], ranks=[0,1,1])

    result = [item for sublist in result for item in sublist]

    y = len(result_list)
    
    player_frame_old = player_frame

    for item in result_list:
        player_frame = player_frame[player_frame['name'] != item]
    
    for i in range(0,y):
        player_frame  = player_frame.append({'name': result_list[i], \
                                         'mu': result[i].mu, 'sigma': result[i].sigma}, ignore_index=True)
    
    #added 4.4.2021: should prevent losers in multiway matches from upping their rankings because of competition
    for item in result_list[2:]:
        if get_rating(item, player_frame).mu > get_rating(item, player_frame_old).mu:
            player_frame.loc[(player_frame.name == item),'mu'] = get_rating(item, player_frame_old).mu

    return player_frame

def victory_fourwaytag(result_list, player_frame):
    '''
    updates player_frame after a tag matchup that is not a tie
    '''
    
    x1 = Rating(get_rating(result_list[0], player_frame),)
    x2 = Rating(get_rating(result_list[1], player_frame),)

    y1 = Rating(get_rating(result_list[2], player_frame),)
    y2 = Rating(get_rating(result_list[3], player_frame),)
    
    z1 = Rating(get_rating(result_list[4], player_frame),)
    z2 = Rating(get_rating(result_list[5], player_frame),)
    
    v1 = Rating(get_rating(result_list[6], player_frame),)
    v2 = Rating(get_rating(result_list[7], player_frame),)

    rate_pt2 = [0, 1, 1, 1]
    
    result = rate([(x1,x2), (y1,y2), (z1, z2), (v1, v2)], ranks=[0,1,1,1])

    result = [item for sublist in result for item in sublist]

    y = len(result_list)
    
    player_frame_old = player_frame

    for item in result_list:
        player_frame = player_frame[player_frame['name'] != item]
    
    for i in range(0,y):
        player_frame  = player_frame.append({'name': result_list[i], \
                                         'mu': result[i].mu, 'sigma': result[i].sigma}, ignore_index=True)
    
    #added 4.4.2021: should prevent losers in multiway matches from upping their rankings because of competition
    for item in result_list[2:]:
        if get_rating(item, player_frame).mu > get_rating(item, player_frame_old).mu:
            player_frame.loc[(player_frame.name == item),'mu'] = get_rating(item, player_frame_old).mu

    return player_frame

def victory_tag6(result_list, player_frame):
    '''
    updates player_frame after a tag matchup that is not a tie
    '''
    
    x1 = Rating(get_rating(result_list[0], player_frame),)
    x2 = Rating(get_rating(result_list[1], player_frame),)
    x3 = Rating(get_rating(result_list[2], player_frame),)

    y1 = Rating(get_rating(result_list[3], player_frame),)
    y2 = Rating(get_rating(result_list[4], player_frame),)
    y3 = Rating(get_rating(result_list[5], player_frame),)

    rate_pt2 = [0, 1]
    
    result = rate([(x1,x2,x3), (y1,y2,y3)], ranks=[0,1])

    result = [item for sublist in result for item in sublist]

    y = len(result_list)
    
    player_frame_old = player_frame

    for item in result_list:
        player_frame = player_frame[player_frame['name'] != item]
    
    for i in range(0,y):
        player_frame  = player_frame.append({'name': result_list[i], \
                                         'mu': result[i].mu, 'sigma': result[i].sigma}, ignore_index=True)
    
    #added 4.4.2021: should prevent losers in multiway matches from upping their rankings because of competition
    for item in result_list[3:]:
        if get_rating(item, player_frame).mu > get_rating(item, player_frame_old).mu:
            player_frame.loc[(player_frame.name == item),'mu'] = get_rating(item, player_frame_old).mu

    return player_frame

def victory_tag8(result_list, player_frame):
    '''
    updates player_frame after a tag matchup that is not a tie
    '''
    
    x1 = Rating(get_rating(result_list[0], player_frame),)
    x2 = Rating(get_rating(result_list[1], player_frame),)
    x3 = Rating(get_rating(result_list[2], player_frame),)
    x4 = Rating(get_rating(result_list[3], player_frame),)

    y1 = Rating(get_rating(result_list[4], player_frame),)
    y2 = Rating(get_rating(result_list[5], player_frame),)
    y3 = Rating(get_rating(result_list[6], player_frame),)
    y4 = Rating(get_rating(result_list[7], player_frame),)

    rate_pt2 = [0, 1]

    
    result = rate([(x1,x2,x3,x4), (y1,y2,y3,y4)], ranks=[0,1])

    result = [item for sublist in result for item in sublist]

    y = len(result_list)
    
    player_frame_old = player_frame

    for item in result_list:
        player_frame = player_frame[player_frame['name'] != item]
    
    for i in range(0,y):
        player_frame  = player_frame.append({'name': result_list[i], \
                                         'mu': result[i].mu, 'sigma': result[i].sigma}, ignore_index=True)
    
    #added 4.4.2021: should prevent losers in multiway matches from upping their rankings because of competition
    for item in result_list[4:]:
        if get_rating(item, player_frame).mu > get_rating(item, player_frame_old).mu:
            player_frame.loc[(player_frame.name == item),'mu'] = get_rating(item, player_frame_old).mu

    return player_frame

def victory_tag10(result_list, player_frame):
    '''
    updates player_frame after a tag matchup that is not a tie
    '''
    
    x1 = Rating(get_rating(result_list[0], player_frame),)
    x2 = Rating(get_rating(result_list[1], player_frame),)
    x3 = Rating(get_rating(result_list[2], player_frame),)
    x4 = Rating(get_rating(result_list[3], player_frame),)
    x5 = Rating(get_rating(result_list[4], player_frame),)

    y1 = Rating(get_rating(result_list[5], player_frame),)
    y2 = Rating(get_rating(result_list[6], player_frame),)
    y3 = Rating(get_rating(result_list[7], player_frame),)
    y4 = Rating(get_rating(result_list[8], player_frame),)
    y5 = Rating(get_rating(result_list[9], player_frame),)

    rate_pt2 = [0, 1]
    
    result = rate([(x1,x2,x3,x4,x5), (y1,y2,y3,y4,y5)], ranks=[0,1])

    result = [item for sublist in result for item in sublist]

    y = len(result_list)
    
    player_frame_old = player_frame

    for item in result_list:
        player_frame = player_frame[player_frame['name'] != item]
    
    for i in range(0,y):
        player_frame  = player_frame.append({'name': result_list[i], \
                                         'mu': result[i].mu, 'sigma': result[i].sigma}, ignore_index=True)
    
    #added 4.4.2021: should prevent losers in multiway matches from upping their rankings because of competition
    for item in result_list[5:]:
        if get_rating(item, player_frame).mu > get_rating(item, player_frame_old).mu:
            player_frame.loc[(player_frame.name == item),'mu'] = get_rating(item, player_frame_old).mu

    return player_frame

def victory_tag12(result_list, player_frame):
    '''
    updates player_frame after a tag matchup that is not a tie
    '''
    
    x1 = Rating(get_rating(result_list[0], player_frame),)
    x2 = Rating(get_rating(result_list[1], player_frame),)
    x3 = Rating(get_rating(result_list[2], player_frame),)
    x4 = Rating(get_rating(result_list[3], player_frame),)
    x5 = Rating(get_rating(result_list[4], player_frame),)
    x6 = Rating(get_rating(result_list[5], player_frame),)

    y1 = Rating(get_rating(result_list[6], player_frame),)
    y2 = Rating(get_rating(result_list[7], player_frame),)
    y3 = Rating(get_rating(result_list[8], player_frame),)
    y4 = Rating(get_rating(result_list[9], player_frame),)
    y5 = Rating(get_rating(result_list[10], player_frame),)
    y6 = Rating(get_rating(result_list[11], player_frame),)

    rate_pt2 = [0, 1]
    
    result = rate([(x1,x2,x3,x4,x5,x6), (y1,y2,y3,y4,y5,y6)], ranks=[0,1])

    result = [item for sublist in result for item in sublist]

    y = len(result_list)
    
    player_frame_old = player_frame

    for item in result_list:
        player_frame = player_frame[player_frame['name'] != item]
    
    for i in range(0,y):
        player_frame  = player_frame.append({'name': result_list[i], \
                                         'mu': result[i].mu, 'sigma': result[i].sigma}, ignore_index=True)
    
    #added 4.4.2021: should prevent losers in multiway matches from upping their rankings because of competition
    for item in result_list[6:]:
        if get_rating(item, player_frame).mu > get_rating(item, player_frame_old).mu:
            player_frame.loc[(player_frame.name == item),'mu'] = get_rating(item, player_frame_old).mu

    return player_frame

def victory_handicap3on1(result_list, player_frame, date):
    '''
    updates player_frame after a tag matchup that is not a tie
    '''
    
    if date == pd.to_datetime('2020-08-27').normalize():
        
        x1 = Rating(get_rating('Big Swole', player_frame),)
        x2 = Rating(25,3)
        x3 = Rating(25,3)

        y1 = Rating(get_rating('Dr. Britt Baker DMD', player_frame),)
        y2 = Rating(get_rating('Penelope Ford', player_frame),)
        y3 = Rating(get_rating('Reba', player_frame),)

        rate_pt2 = [0, 1]
    
        result = rate([(x1,x2,x3), (y1,y2,y3)], ranks=[0,1])

        result = [item for sublist in result for item in sublist]

        y = len(result_list)
    
        player_frame_old = player_frame

        for item in result_list:
            player_frame = player_frame[player_frame['name'] != item]

        for i in range(0,y):
            player_frame  = player_frame.append({'name': result_list[i], \
                                             'mu': result[i].mu, 'sigma': result[i].sigma}, ignore_index=True)
    
        #added 4.4.2021: should prevent losers in multiway matches from upping their rankings because of competition
        for item in result_list[2:]:
            if get_rating(item, player_frame).mu > get_rating(item, player_frame_old).mu:
                player_frame.loc[(player_frame.name == item),'mu'] = get_rating(item, player_frame_old).mu

    return player_frame

def victory_handicap2on1(result_list, player_frame, date):
    '''
    updates player_frame after a tag matchup that is not a tie
    '''
    
    if date == pd.to_datetime('2020-08-12').normalize():
        
        x1 = Rating(get_rating('Lance Archer', player_frame),)
        x2 = Rating(25,1)

        y1 = Rating(get_rating('Jessy Sorensen', player_frame),)
        y2 = Rating(get_rating('Jon Cruz', player_frame),)

        rate_pt2 = [0, 1]
    
        result = rate([(x1,x2), (y1,y2)], ranks=[0,1])

        result = [item for sublist in result for item in sublist]

        y = len(result_list)
    
        player_frame_old = player_frame

        for item in result_list:
            player_frame = player_frame[player_frame['name'] != item]

        for i in range(0,y):
            player_frame  = player_frame.append({'name': result_list[i], \
                                             'mu': result[i].mu, 'sigma': result[i].sigma}, ignore_index=True)
    
        #added 4.4.2021: should prevent losers in multiway matches from upping their rankings because of competition
        for item in result_list[2:]:
            if get_rating(item, player_frame).mu > get_rating(item, player_frame_old).mu:
                player_frame.loc[(player_frame.name == item),'mu'] = get_rating(item, player_frame_old).mu
                
    if date == pd.to_datetime('2020-07-02').normalize():
        
        x1 = Rating(get_rating('Nyla Rose', player_frame),)
        x2 = Rating(25,1)

        y1 = Rating(get_rating('Kenzie Paige', player_frame),)
        y2 = Rating(get_rating('KiLynn King', player_frame),)

        rate_pt2 = [0, 1]
    
        result = rate([(x1,x2), (y1,y2)], ranks=[0,1])

        result = [item for sublist in result for item in sublist]

        y = len(result_list)
    
        player_frame_old = player_frame

        for item in result_list:
            player_frame = player_frame[player_frame['name'] != item]

        for i in range(0,y):
            player_frame  = player_frame.append({'name': result_list[i], \
                                             'mu': result[i].mu, 'sigma': result[i].sigma}, ignore_index=True)
    
        #added 4.4.2021: should prevent losers in multiway matches from upping their rankings because of competition
        for item in result_list[2:]:
            if get_rating(item, player_frame).mu > get_rating(item, player_frame_old).mu:
                player_frame.loc[(player_frame.name == item),'mu'] = get_rating(item, player_frame_old).mu
                
    if date == pd.to_datetime('2020-03-04').normalize():
        
        x1 = Rating(get_rating('Chris Jericho', player_frame),)
        x2 = Rating(get_rating('Sammy Guevara', player_frame),)
        
        y1 = Rating(get_rating('Darby Allin', player_frame),)
        y2 = Rating(25,1)
        
        print(x1)

        rate_pt2 = [0, 1]
    
        result = rate([(x1,x2), (y1,y2)], ranks=[0,1])

        result = [item for sublist in result for item in sublist]

        y = len(result_list)
    
        player_frame_old = player_frame

        for item in result_list:
            player_frame = player_frame[player_frame['name'] != item]

        for i in range(0,y):
            player_frame  = player_frame.append({'name': result_list[i], \
                                             'mu': result[i].mu, 'sigma': result[i].sigma}, ignore_index=True)
    
        #added 4.4.2021: should prevent losers in multiway matches from upping their rankings because of competition
        for item in result_list[2:]:
            if get_rating(item, player_frame).mu > get_rating(item, player_frame_old).mu:
                player_frame.loc[(player_frame.name == item),'mu'] = get_rating(item, player_frame_old).mu

    return player_frame

### does the deal

For every day since May 1, 2019, checks if there was at least one AEW match that happened that day in the data. If there was, adds each match to the larger results frame. Adds player to the results frame if they are new. If the day is December 31, performs rankings reset.

In [471]:
player_frame = create_player_frame()
macro_player_frame = create_macro_player_frame()

pandas_today = pd.to_datetime('today').normalize()
x = pd.to_datetime('2019-05-01').normalize()

while x < pandas_today:
    
    #optionally can add (all_matches['matchtype'] == 'singles') for specific divisions only
    day_matches = all_matches[(all_matches['Date'] == x)]
    day_results = [item for item in day_matches.result]

    for index, row in day_matches.iterrows():
        placement = [item for item in [row.win1, row.win2, row.win3, row.win4, row.win5, row.win6,\
                                       row.lose1, row.lose2, row.lose3, row.lose4, row.lose5, row.lose6,\
                                      row.lose7, row.lose8] if str(item) != 'nan']
        placement = [alias_collapser(item) if item in aliases else item for item in placement]

        for item in placement:
            if item not in set(player_frame.name):
                player_frame = player_add(item, player_frame)
        if row.matchtype in ['singles', 'threeway', 'fourway', 'sixway', 'nineway']:
            player_frame = victory_singles(placement, player_frame)
        if row.matchtype in ['draw']:
            player_frame = victory_draw(placement, player_frame)
        if row.matchtype in ['tag']:
            player_frame = victory_tag(placement, player_frame)
        if row.matchtype in ['tag6']:
            player_frame = victory_tag6(placement, player_frame)
        if row.matchtype in ['tag8']:
            player_frame = victory_tag8(placement, player_frame)
        if row.matchtype in ['tag10']:
            player_frame = victory_tag10(placement, player_frame)
        if row.matchtype in ['tag12']:
            player_frame = victory_tag12(placement, player_frame)
        if row.matchtype in ['threewaytag']:
            player_frame = victory_threewaytag(placement, player_frame)
        if row.matchtype in ['fourwaytag']:
            player_frame = victory_fourwaytag(placement, player_frame)
        if row.matchtype in ['handicap3on1']:
            player_frame = victory_handicap3on1(placement, player_frame,x)
        if row.matchtype in ['handicap2on1']:
            player_frame = victory_handicap2on1(placement, player_frame,x)
    
    intermediary = player_frame
    intermediary['Date'] = x
    intermediary['Result'] = ''
    
    for index, row in intermediary.iterrows():
        for item in day_results:
            if str(row['name']) in item:
                intermediary.loc[index, 'Result'] = item
                
    macro_player_frame = pd.concat([macro_player_frame, intermediary])
    
    if (x.month == 12) & (x.day == 31):    
        player_frame = player_frame[~(player_frame.sigma == 8.33)]
        player_frame.mu = [25 + ((i - 25) / 2) for i in player_frame.mu]
        player_frame.sigma = [8.33 for i in player_frame.sigma]
    
    x = x + pd.DateOffset(1)

trueskill.Rating(mu=31.150, sigma=6.522)


In [472]:
macro_player_frame = macro_player_frame.groupby(['name', 'mu', 'sigma']).aggregate({"Date": "min", "Result": "max"})

In [473]:
x = macro_player_frame.groupby(by=["name"], dropna=False).count()
matchcounts = pd.DataFrame(x['Date'])

player_frame = player_frame.sort_values('name', ascending=True)

In [474]:
player_frame[player_frame.sigma < 8.333].sort_values('mu', ascending=False).to_csv('ranking_output_singles_' + today + '.csv', index=False)


In [475]:
player_frame[player_frame.sigma < 8.333].sort_values('mu', ascending=False).to_csv('ranking_output_' + today + '.csv', index=False)
macro_player_frame.sort_values('mu', ascending=False).to_csv('macro_output_' + today + '.csv')

macro_player_frame.sort_values('mu', ascending=False).to_csv('macro_output.csv')

In [476]:
macro_player_frame.sort_values('Date').filter(like = 'Nyla Rose', axis=0)

Date  \
name      mu        sigma                 
Nyla Rose 23.193658 5.429410 2019-05-25   
          21.995731 4.577329 2019-06-29   
          21.194612 4.339793 2019-10-02   
          22.678838 4.083843 2019-10-22   
          24.038802 3.983094 2019-11-08   
          25.341580 3.803664 2019-11-13   
          25.937549 3.646443 2019-12-04   
          25.423307 3.498637 2019-12-31   
          24.445156 5.464429 2020-01-01   
          26.302210 5.081015 2020-01-28   
          28.623827 4.660875 2020-01-29   
          32.229625 4.176263 2020-02-12   
          33.656910 3.903868 2020-02-29   
          34.670649 3.800202 2020-03-11   
          35.146985 3.701819 2020-05-06   
          36.132640 3.556906 2020-05-20   
          34.252171 3.252907 2020-05-23   
          34.612937 3.152531 2020-06-03   
          35.462702 3.051342 2020-06-10   
          34.455621 3.004853 2020-08-03   
          34.467308 3.000069 2020-08-25   
          34.905487 2.925192 2020-09-09   
          34.974297 2.903918 2020-09-29   
          34.979676 2.902153 2020-10-13   
          35.001926 2.892905 2020-11-04   
          34.084431 2.733721 2020-11-07   
          34.118837 2.722395 2020-12-08   
          34.125083 2.720525 2020-12-22   
          29.562542 8.330000 2021-01-01   
          32.095290 7.453907 2021-01-05   
          33.747864 6.867457 2021-02-09   
          36.860948 5.910199 2021-02-22   
          39.771897 5.185591 2021-02-24   
          41.140458 4.766502 2021-03-01   
          38.151631 4.278689 2021-03-03   
          37.646525 4.205254 2021-03-10   
          34.785057 3.771440 2021-03-24   
          36.533618 3.650045 2021-03-31   
          36.284813 3.621717 2021-04-09   
          36.368566 3.588009 2021-04-13   
          35.962822 3.514200 2021-04-19   
          35.564668 3.451365 2021-04-26   
          35.678590 3.405287 2021-05-03   

                                                                         Result  
name      mu        sigma                                                        
Nyla Rose 23.193658 5.429410  Dr. Britt Baker DMD defeats Awesome Kong  and ...  
          21.995731 4.577329   Riho defeats Nyla Rose and Yuka Sakazaki (13:02)  
          21.194612 4.339793             (vakant)Riho defeats Nyla Rose (13:26)  
          22.678838 4.083843  Nyla Rose defeats Leva Bates (w/Peter Avalon) ...  
          24.038802 3.983094  Leva Bates & Nyla Rose defeat Shalandra Royal ...  
          25.341580 3.803664               Nyla Rose defeats Dani Jordyn (1:34)  
          25.937549 3.646443  Nyla Rose defeats Leva Bates (w/Peter Avalon) ...  
          25.423307 3.498637                     (vakant)Riho defeats Nyla Rose  
          24.445156 5.464429  Riho  defeats Dr. Britt Baker DMD and Hikaru S...  
          26.302210 5.081015                    Nyla Rose defeats Shanna (8:05)  
          28.623827 4.660875                 Nyla Rose defeats Big Swole (8:44)  
          32.229625 4.176263                    Nyla Rose defeats Riho  (13:07)  
          33.656910 3.903868         Nyla Rose  defeats Kris Statlander (12:49)  
          34.670649 3.800202  Bea Priestley & Nyla Rose defeat Hikaru Shida ...  
          35.146985 3.701819              Nyla Rose defeats Kenzie Paige (2:19)  
          36.132640 3.556906  Dr. Britt Baker DMD & Nyla Rose defeat Hikaru ...  
          34.252171 3.252907            Hikaru Shida defeats Nyla Rose  (16:34)  
          34.612937 3.152531                 Nyla Rose defeats Big Swole (6:33)  
          35.462702 3.051342  Nyla Rose defeats Kenzie Paige & KiLynn King (...  
          34.455621 3.004853  Anna Jay & Tay Conti defeat Ariane Andrew & Ny...  
          34.467308 3.000069  Nyla Rose (w/Vickie Guerrero) defeats KiLynn K...  
          34.905487 2.925192  Nyla Rose (w/Vickie Guerrero) defeats Tay Cont...  
          34.974297 2.903918  Nyla Rose (w/Vickie Guerrero) defeats Rache Ch...  
          34.979676 2.902153  Nyla Rose (w/Vickie

## Double-macro frame (all date comparison)

In [477]:
x = pd.read_csv('macro_output.csv')
x = x.sort_values(['name', 'Date'])

x.Date

3166    2021-01-19
2761    2021-02-23
2945    2021-03-02
3042    2021-03-09
3278    2021-03-29
           ...    
4073    2020-09-04
3393    2021-01-01
3548    2021-01-12
4134    2021-01-19
4222    2021-04-12
Name: Date, Length: 4726, dtype: object

In [478]:
x = pd.read_csv('macro_output.csv')
x = x.sort_values(['name', 'Date'])

double_macro = pd.DataFrame(columns=['name', 'mu', 'sigma', 'Date'])

zz = 0
print(len(set(x.name)))

for guy in set(x.name):
    print(zz)
    print(guy)
    y = x[x.name == guy]
    y = y.reset_index()
    y = y[['name', 'mu', 'sigma', 'Date']]
    y['Date'] = pd.to_datetime(y.Date, format='%Y-%m-%d')
    
    date1 = y.Date[0] #first appearance
    date2 = y.Date[len(y) - 1] #last appearance
    
    cutoff = date2 + relativedelta(months=+3)
    
    print(date1)
    print(cutoff)
    
    check_date = date1
    most_recent_match = date1
    
    while (check_date < cutoff) & (check_date < pandas_today):
        all_dates = list(y.Date)
        
        if (check_date not in all_dates) & (check_date < most_recent_match + pd.DateOffset(90)):
            y = y.append({'name': guy, 'mu': None, 'sigma': None, 'Date': check_date}, ignore_index=True)
            
        if (check_date in all_dates) & (check_date != '2021-01-01 00:00:00'):
            most_recent_match = check_date
            
        check_date = check_date + pd.DateOffset(1)
        
    y = y.sort_values('Date')
    y = y.fillna(method='ffill')
    
    double_macro = pd.concat([double_macro, y])
    
    zz = zz + 1

350
0
Freya States
2020-12-08 00:00:00
2021-04-01 00:00:00
1
Azrieal
2021-03-06 00:00:00
2021-06-09 00:00:00
2
QT Marshall
2019-10-29 00:00:00
2021-08-05 00:00:00
3
Terrell Hughes
2020-11-10 00:00:00
2021-08-04 00:00:00
4
El Lindaman
2019-05-25 00:00:00
2021-04-01 00:00:00
5
Ganon Jones
2021-04-27 00:00:00
2021-08-03 00:00:00
6
Cody Rhodes
2019-05-25 00:00:00
2021-08-05 00:00:00
7
Reka Tehaka
2021-04-09 00:00:00
2021-08-10 00:00:00
8
Alex Chamberlain
2020-08-11 00:00:00
2021-04-01 00:00:00
9
El Australiano
2021-01-12 00:00:00
2021-04-19 00:00:00
10
Diamante
2020-01-21 00:00:00
2021-08-04 00:00:00
11
Stone Rockwell
2021-04-13 00:00:00
2021-07-19 00:00:00
12
Lady Frost
2020-11-24 00:00:00
2021-04-01 00:00:00
13
Griff Garrison
2020-06-16 00:00:00
2021-08-05 00:00:00
14
Aaron Solow
2020-07-21 00:00:00
2021-07-28 00:00:00
15
Serpentico
2020-05-19 00:00:00
2021-08-10 00:00:00
16
Queen Aminata
2021-04-13 00:00:00
2021-07-20 00:00:00
17
KENTA
2021-02-10 00:00:00
2021-05-10 00:00:00
18
KTB
2020

149
Marty Casaus
2021-04-27 00:00:00
2021-07-27 00:00:00
150
Big Game Leroy
2020-06-02 00:00:00
2021-04-01 00:00:00
151
Bobby Wayward
2020-12-01 00:00:00
2021-04-01 00:00:00
152
Matt Hardy
2020-05-06 00:00:00
2021-08-10 00:00:00
153
Ricky Starks
2020-06-17 00:00:00
2021-08-04 00:00:00
154
JJ Garrett
2021-02-23 00:00:00
2021-06-02 00:00:00
155
Shaq
2021-03-03 00:00:00
2021-06-03 00:00:00
156
Spencer Slade
2021-04-13 00:00:00
2021-08-04 00:00:00
157
Steff MK
2021-04-20 00:00:00
2021-07-20 00:00:00
158
Alex Reynolds
2019-10-16 00:00:00
2021-08-10 00:00:00
159
Joe Alonzo
2020-03-24 00:00:00
2021-04-01 00:00:00
160
Maxx Stardom
2020-09-15 00:00:00
2021-04-01 00:00:00
161
Jazmin Allure
2020-12-22 00:00:00
2021-07-06 00:00:00
162
James Tapia
2020-12-29 00:00:00
2021-04-01 00:00:00
163
George South Jr.
2020-12-01 00:00:00
2021-04-01 00:00:00
164
Simon Lotto
2020-04-06 00:00:00
2021-04-01 00:00:00
165
Ivelisse
2020-07-22 00:00:00
2021-06-02 00:00:00
166
Lindsay Snow
2020-11-17 00:00:00
2021-04-

295
Mark Sterling
2020-09-02 00:00:00
2021-04-01 00:00:00
296
Tony Vincita
2020-10-27 00:00:00
2021-05-09 00:00:00
297
Wardlow
2020-02-19 00:00:00
2021-08-05 00:00:00
298
Awesome Kong
2019-05-25 00:00:00
2021-04-01 00:00:00
299
Ariel Dominguez
2020-11-06 00:00:00
2021-04-01 00:00:00
300
Liam Gray
2021-04-26 00:00:00
2021-08-04 00:00:00
301
Shazza McKenzie
2019-11-08 00:00:00
2020-04-01 00:00:00
302
Darius Martin
2020-10-27 00:00:00
2021-06-02 00:00:00
303
AJ Kirsch
2021-01-19 00:00:00
2021-04-19 00:00:00
304
Jimmy Havoc
2019-06-29 00:00:00
2021-04-01 00:00:00
305
Powerhouse Hobbs
2020-07-07 00:00:00
2021-08-04 00:00:00
306
Joey O'Riley
2020-11-17 00:00:00
2021-04-01 00:00:00
307
Thunder Rosa
2020-09-02 00:00:00
2021-08-10 00:00:00
308
Orange Cassidy
2019-10-30 00:00:00
2021-08-03 00:00:00
309
Katalina Perez
2020-10-27 00:00:00
2021-07-12 00:00:00
310
Alex Gracia
2020-09-29 00:00:00
2021-07-05 00:00:00
311
Brian Pillman Jr.
2020-07-07 00:00:00
2021-08-05 00:00:00
312
Kenzie Paige
2020-0

In [479]:
double_macro

,name,mu,sigma,Date
0,Freya States,16.858482,6.253237,2020-12-08
3,Freya States,16.858482,6.253237,2020-12-09
4,Freya States,16.858482,6.253237,2020-12-10
5,Freya States,16.858482,6.253237,2020-12-11
6,Freya States,16.858482,6.253237,2020-12-12
...,...,...,...,...
23,Kit Sackett,18.760216,6.534945,2021-05-06
24,Kit Sackett,18.760216,6.534945,2021-05-07
25,Kit Sackett,18.760216,6.534945,2021-05-08
26,Kit Sackett,18.760216,6.534945,2021-05-09


In [480]:
double_macro.sort_values('name', ascending=False).to_csv('double_macro_' + today + '.csv')
double_macro.sort_values('name', ascending=False).to_csv('double_macro.csv')

In [481]:
double_macro[double_macro.Date == '2020-10-01'].sort_values('mu', ascending=False)

,name,mu,sigma,Date
376,Jon Moxley,45.150648,3.336279,2020-10-01
192,Brodie Lee,39.441999,3.392517,2020-10-01
514,Cody Rhodes,36.651481,2.632785,2020-10-01
512,Kenny Omega,36.252821,3.513333,2020-10-01
198,Lance Archer,36.236986,3.512906,2020-10-01
...,...,...,...,...
151,KiLynn King,11.053680,4.257433,2020-10-01
456,Peter Avalon,9.451552,3.711646,2020-10-01
202,Lee Johnson,9.272073,3.858739,2020-10-01
201,Shawn Dean,7.317298,3.893303,2020-10-01


In [482]:
date1 = pd.to_datetime('2019-05-26').normalize()

while date1 < pandas_today:
    z = double_macro[double_macro.Date == date1]
    z = z.sort_values('mu', ascending=False)
    z = z.reset_index()
    bottom = len(z)
    
    print(date1, '|', z.name[0], '|', z.name[bottom - 1])
    
    date1 = date1 + pd.DateOffset(1)
    


2019-05-26 00:00:00 | Dr. Britt Baker DMD | Kenny Omega
2019-05-27 00:00:00 | Dr. Britt Baker DMD | Kenny Omega
2019-05-28 00:00:00 | Dr. Britt Baker DMD | Kenny Omega
2019-05-29 00:00:00 | Dr. Britt Baker DMD | Kenny Omega
2019-05-30 00:00:00 | Dr. Britt Baker DMD | Kenny Omega
2019-05-31 00:00:00 | Dr. Britt Baker DMD | Kenny Omega
2019-06-01 00:00:00 | Dr. Britt Baker DMD | Kenny Omega
2019-06-02 00:00:00 | Dr. Britt Baker DMD | Kenny Omega
2019-06-03 00:00:00 | Dr. Britt Baker DMD | Kenny Omega
2019-06-04 00:00:00 | Dr. Britt Baker DMD | Kenny Omega
2019-06-05 00:00:00 | Dr. Britt Baker DMD | Kenny Omega
2019-06-06 00:00:00 | Dr. Britt Baker DMD | Kenny Omega
2019-06-07 00:00:00 | Dr. Britt Baker DMD | Kenny Omega
2019-06-08 00:00:00 | Dr. Britt Baker DMD | Kenny Omega
2019-06-09 00:00:00 | Dr. Britt Baker DMD | Kenny Omega
2019-06-10 00:00:00 | Dr. Britt Baker DMD | Kenny Omega
2019-06-11 00:00:00 | Dr. Britt Baker DMD | Kenny Omega
2019-06-12 00:00:00 | Dr. Britt Baker DMD | Kenn

2019-11-23 00:00:00 | Chris Jericho | Brandon Cutler
2019-11-24 00:00:00 | Chris Jericho | Brandon Cutler
2019-11-25 00:00:00 | Chris Jericho | Brandon Cutler
2019-11-26 00:00:00 | Chris Jericho | Big Swole
2019-11-27 00:00:00 | Chris Jericho | Big Swole
2019-11-28 00:00:00 | Chris Jericho | Big Swole
2019-11-29 00:00:00 | Chris Jericho | Big Swole
2019-11-30 00:00:00 | Chris Jericho | Big Swole
2019-12-01 00:00:00 | Chris Jericho | Big Swole
2019-12-02 00:00:00 | Chris Jericho | Big Swole
2019-12-03 00:00:00 | Chris Jericho | Brandon Cutler
2019-12-04 00:00:00 | Chris Jericho | Brandon Cutler
2019-12-05 00:00:00 | Chris Jericho | Brandon Cutler
2019-12-06 00:00:00 | Chris Jericho | Brandon Cutler
2019-12-07 00:00:00 | Chris Jericho | Brandon Cutler
2019-12-08 00:00:00 | Chris Jericho | Brandon Cutler
2019-12-09 00:00:00 | Chris Jericho | Brandon Cutler
2019-12-10 00:00:00 | Chris Jericho | Brandon Cutler
2019-12-11 00:00:00 | Chris Jericho | Brandon Cutler
2019-12-12 00:00:00 | Chris 

2020-05-14 00:00:00 | Jon Moxley | Brandon Cutler
2020-05-15 00:00:00 | Jon Moxley | Brandon Cutler
2020-05-16 00:00:00 | Jon Moxley | Brandon Cutler
2020-05-17 00:00:00 | Jon Moxley | Brandon Cutler
2020-05-18 00:00:00 | Jon Moxley | Brandon Cutler
2020-05-19 00:00:00 | Jon Moxley | Brandon Cutler
2020-05-20 00:00:00 | Jon Moxley | Brandon Cutler
2020-05-21 00:00:00 | Jon Moxley | Brandon Cutler
2020-05-22 00:00:00 | Jon Moxley | Brandon Cutler
2020-05-23 00:00:00 | Jon Moxley | Brandon Cutler
2020-05-24 00:00:00 | Jon Moxley | Brandon Cutler
2020-05-25 00:00:00 | Jon Moxley | Brandon Cutler
2020-05-26 00:00:00 | Jon Moxley | Brandon Cutler
2020-05-27 00:00:00 | Jon Moxley | Brandon Cutler
2020-05-28 00:00:00 | Jon Moxley | Brandon Cutler
2020-05-29 00:00:00 | Jon Moxley | Brandon Cutler
2020-05-30 00:00:00 | Jon Moxley | Brandon Cutler
2020-05-31 00:00:00 | Jon Moxley | Brandon Cutler
2020-06-01 00:00:00 | Jon Moxley | Brandon Cutler
2020-06-02 00:00:00 | Jon Moxley | Brandon Cutler


2020-11-13 00:00:00 | Jon Moxley | Shawn Dean
2020-11-14 00:00:00 | Jon Moxley | Shawn Dean
2020-11-15 00:00:00 | Jon Moxley | Shawn Dean
2020-11-16 00:00:00 | Jon Moxley | Shawn Dean
2020-11-17 00:00:00 | Jon Moxley | Shawn Dean
2020-11-18 00:00:00 | Jon Moxley | Shawn Dean
2020-11-19 00:00:00 | Jon Moxley | Shawn Dean
2020-11-20 00:00:00 | Jon Moxley | Shawn Dean
2020-11-21 00:00:00 | Jon Moxley | Shawn Dean
2020-11-22 00:00:00 | Jon Moxley | Shawn Dean
2020-11-23 00:00:00 | Jon Moxley | Shawn Dean
2020-11-24 00:00:00 | Jon Moxley | Shawn Dean
2020-11-25 00:00:00 | Jon Moxley | Shawn Dean
2020-11-26 00:00:00 | Jon Moxley | Shawn Dean
2020-11-27 00:00:00 | Jon Moxley | Shawn Dean
2020-11-28 00:00:00 | Jon Moxley | Shawn Dean
2020-11-29 00:00:00 | Jon Moxley | Shawn Dean
2020-11-30 00:00:00 | Jon Moxley | Shawn Dean
2020-12-01 00:00:00 | Jon Moxley | Shawn Dean
2020-12-02 00:00:00 | Jon Moxley | Shawn Dean
2020-12-03 00:00:00 | Jon Moxley | Shawn Dean
2020-12-04 00:00:00 | Jon Moxley |

2021-05-06 00:00:00 | Darby Allin | Fuego Del Sol
2021-05-07 00:00:00 | Darby Allin | Fuego Del Sol
2021-05-08 00:00:00 | Darby Allin | Fuego Del Sol
2021-05-09 00:00:00 | Darby Allin | Fuego Del Sol
2021-05-10 00:00:00 | Darby Allin | Fuego Del Sol
